# 0. Setup

## python version

In [7]:
from platform import python_version
print(python_version())

3.7.9


## Display Size

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Librerie

In [9]:
# Scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import mechanicalsoup as ms
import scrapy
import selenium
import lxml
import re
import requests
import pandas as pd
from pandas import compat
import json
from requests import get
import pprint

# Matching
import Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Record Linkage
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonetic
#import html5lib

## Setup Selenium

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 1. `pokemondb` Tables (Gen VIII All-Things e National Pokédex) 

## NATIONAL POKEDEX e SPRITES: Pokemon list with stats (id, nome, tipo, total, hp, attack, defense, sp. atk., sp. def, speed)

- *Il pokédex nazionale è quello che cointiene TUTTI i pokemon usciti nelle diverse generazioni (in totale 905)*

Url della risorsa, HTTPResponse Object e BeautifulSoupObject della pagina

In [52]:
pokedex_URL = "https://pokemondb.net/pokedex/all"
pokedex_page = requests.get(pokedex_URL)

In [53]:
pokedex_soup = BeautifulSoup(pokedex_page.content, features="html.parser")

In [54]:
pokedex_table = pokedex_soup.find(id="pokedex")

In [55]:
pokedex_df = pd.read_html(str(pokedex_table))[0]

In [56]:
pokedex_name_split = pokedex_df['Name'].str.split(n = 1, expand = True)
pokedex_df['Name'] = pokedex_name_split[0]
pokedex_df.insert(2, "Variant", pokedex_name_split[1])
#pokedex_df['Type'] = pokedex_df['Type'].replace(' ', ', ', regex = True)

In [57]:
pokedex_type_split = pokedex_df['Type'].str.split(n = 1, expand = True)
pokedex_df.drop('Type', inplace = True, axis = 1)
pokedex_df.insert(2, "Type1", pokedex_type_split[0])
pokedex_df.insert(3, "Type2", pokedex_type_split[1])

`#` indica il numero nel pokedex nazionale dell'ottava generazione
- Vedi: https://pokemondb.net/pokedex/national

In [58]:
pokedex_df

,#,Name,Type1,Type2,Variant,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,None,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,None,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,None,525,80,82,83,100,100,80
3,3,Venusaur,Grass,Poison,Mega Venusaur,625,80,100,123,122,120,80
4,4,Charmander,Fire,None,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion,Water,Ghost,Female,530,120,92,65,100,75,78
1071,903,Sneasler,Poison,Fighting,None,510,80,130,60,40,80,120
1072,904,Overqwil,Dark,Poison,None,510,85,115,95,65,65,85
1073,905,Enamorus,Fairy,Flying,Incarnate Forme,580,74,115,70,135,80,106


### Sprite

In [84]:
pokedex_sprites = pokedex_table.find('tbody').find_all('span', class_ = 'img-fixed icon-pkmn')

In [87]:
len(pokedex_sprites)

1075

In [88]:
pokemon_sprites = []
for pokemon_sprite in pokedex_sprites:
    pokemon_sprites.append(pokemon_sprite['data-src'])

Aggiungo la colonna

In [90]:
pokedex_df['sprite_url'] = pokemon_sprites

In [91]:
pokedex_df

,#,Name,Type1,Type2,Variant,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sprite_url
0,1,Bulbasaur,Grass,Poison,None,318,45,49,49,65,65,45,https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Grass,Poison,None,405,60,62,63,80,80,60,https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Grass,Poison,None,525,80,82,83,100,100,80,https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,Grass,Poison,Mega Venusaur,625,80,100,123,122,120,80,https://img.pokemondb.net/sprites/sword-shield...
4,4,Charmander,Fire,None,None,309,39,52,43,60,50,65,https://img.pokemondb.net/sprites/sword-shield...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,902,Basculegion,Water,Ghost,Female,530,120,92,65,100,75,78,https://img.pokemondb.net/s.png
1071,903,Sneasler,Poison,Fighting,None,510,80,130,60,40,80,120,https://img.pokemondb.net/s.png
1072,904,Overqwil,Dark,Poison,None,510,85,115,95,65,65,85,https://img.pokemondb.net/s.png
1073,905,Enamorus,Fairy,Flying,Incarnate Forme,580,74,115,70,135,80,106,https://img.pokemondb.net/s.png


### Salvataggio

In [92]:
pokedex_json = json.dumps(list(pokedex_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4)

In [133]:
#print(pokedex_json)

Salvataggio

In [94]:
# JSON
with open('./Scraping_datasets/pokemondb/pokedex.json', 'w', encoding='utf8') as outfile:
    outfile.write(pokedex_json)
# CSV
pokedex_df.to_csv('./Scraping_datasets/pokemondb/pokedex.csv', na_rep="NA")

In [25]:
#pokedex_json = pokedex_df.to_json(orient='index', indent = 4)
#print(pokedex_json)

##  Items (Name, Category, Effect)

In [96]:
items_URL = "https://pokemondb.net/item/all"

In [97]:
items_page = requests.get(items_URL)

In [98]:
items_soup = BeautifulSoup(items_page.content, parser = 'html.parser')

In [99]:
items_df = pd.read_html(str(items_soup))[0]

In [100]:
items_df

,Name,Category,Effect
0,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...
1,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...
2,Ability Urge,Battle items,"When used, it activates the Ability of an ally..."
3,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.
4,Absolite,Hold items,Enables Absol to Mega Evolve during battle.
...,...,...,...
955,Yellow Scarf,General items,Raises holder's Tough aspect in a Contest.
956,Yellow Shard,General items,A small yellow shard. It appears to be from so...
957,Zap Plate,Hold items,Increases power of Electric-type moves. Change...
958,Zinc,Medicine,Increases Special Defense EVs by 10.


### Sprites

In [122]:
items_sprites_span = items_soup.find('table',
                              class_='data-table').find('tbody').find_all(
                                  'span', class_='img-fixed icon-item-img')

In [124]:
items_sprites = []
for item_sprite in items_sprites_span:
    items_sprites.append(item_sprite['data-src'])

Aggiungo la colonna

In [126]:
items_df['sprite_url'] = items_sprites

In [127]:
items_df

,Name,Category,Effect,sprite_url
0,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...,https://img.pokemondb.net/s.png
2,Ability Urge,Battle items,"When used, it activates the Ability of an ally...",https://img.pokemondb.net/sprites/items/abilit...
3,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/abomas...
4,Absolite,Hold items,Enables Absol to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/absoli...
...,...,...,...,...
955,Yellow Scarf,General items,Raises holder's Tough aspect in a Contest.,https://img.pokemondb.net/sprites/items/yellow...
956,Yellow Shard,General items,A small yellow shard. It appears to be from so...,https://img.pokemondb.net/sprites/items/yellow...
957,Zap Plate,Hold items,Increases power of Electric-type moves. Change...,https://img.pokemondb.net/sprites/items/zap-pl...
958,Zinc,Medicine,Increases Special Defense EVs by 10.,https://img.pokemondb.net/sprites/items/zinc.png


### Salvataggio

In [129]:
items_json = json.dumps(list(items_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)

In [130]:
# JSON
with open('./Scraping_datasets/pokemondb/items.json', 'w', encoding='utf8') as outfile:
    outfile.write(items_json)
# CSV
items_df.to_csv('./Scraping_datasets/pokemondb/items.csv', na_rep="NA")

## abilities (name, n_pokemon, description, Gen)

In [83]:
abilities_URL = "https://pokemondb.net/ability"
abilities_page = requests.get(abilities_URL)
abilities_soup = BeautifulSoup(abilities_page.content, parser = 'html.parser')
abilities_df = pd.read_html(str(abilities_soup))[0]
abilities_json = json.dumps(list(abilities_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/abilities.json', 'w', encoding='utf8') as outfile:
    outfile.write(abilities_json)
# CSV
abilities_df.to_csv('./Scraping_datasets/abilities.csv', na_rep="NA")

In [79]:
abilities_df

,Name,Pokémon,Description,Gen.
0,Adaptability,16,Powers up moves of the same type.,4
1,Aerilate,2,Turns Normal-type moves into Flying-type moves.,6
2,Aftermath,10,Damages the attacker landing the finishing hit.,4
3,Air Lock,1,Eliminates the effects of weather.,3
4,Analytic,12,Boosts move power when the Pokémon moves last.,5
...,...,...,...,...
261,White Smoke,4,Prevents other Pokémon from lowering its stats.,3
262,Wimp Out,1,Switches out when HP drops below half.,7
263,Wonder Guard,1,Only supereffective moves will hit.,3
264,Wonder Skin,5,Makes status-changing moves more likely to miss.,5


In [132]:
#print(abilities_json)

## Pokémon EVs (Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed)

In [84]:
EV_URL = "https://pokemondb.net/ev/all"
EV_page = requests.get(EV_URL)
EV_soup = BeautifulSoup(EV_page.content, parser = 'html.parser')
EV_df = pd.read_html(str(EV_soup))[0]
EV_json = json.dumps(list(EV_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/EV.json', 'w', encoding='utf8') as outfile:
    outfile.write(EV_json)
# CSV
EV_df.to_csv('./Scraping_datasets/EV.csv', na_rep="NA")

In [85]:
EV_df

,#,Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,NaN,NaN,NaN,1.0,NaN,NaN
1,2,Ivysaur,NaN,NaN,NaN,1.0,1.0,NaN
2,3,Venusaur,NaN,NaN,NaN,2.0,1.0,NaN
3,3,Venusaur Mega Venusaur,NaN,NaN,NaN,2.0,1.0,NaN
4,4,Charmander,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
1071,903,Sneasler,NaN,NaN,NaN,NaN,NaN,NaN
1072,904,Overqwil,NaN,NaN,NaN,NaN,NaN,NaN
1073,905,Enamorus Incarnate Forme,NaN,NaN,NaN,NaN,NaN,NaN
1074,905,Enamorus Therian Forme,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
#print(EV_json)

## Moves

Name
Type
Cat.
Power
Acc.
PP
TM
Effect
Prob. (%))

In [87]:
move_URL = "https://pokemondb.net/move/all"
move_page = requests.get(move_URL)
move_soup = BeautifulSoup(move_page.content, parser = 'html.parser')
move_df = pd.read_html(str(move_soup))[0]
move_json = json.dumps(list(move_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/moves.json', 'w', encoding='utf8') as outfile:
    outfile.write(move_json)
# CSV
move_df.to_csv('./Scraping_datasets/moves.csv', na_rep="NA")

In [88]:
move_df

,Name,Type,Cat.,Power,Acc.,PP,TM,Effect,Prob. (%)
0,"10,000,000 Volt Thunderbolt",Electric,NaN,195,—,1,NaN,Pikachu-exclusive Z-Move. High critical hit ra...,—
1,Absorb,Grass,NaN,20,100,25,NaN,User recovers half the HP inflicted on opponent.,—
2,Accelerock,Rock,NaN,40,100,20,NaN,User attacks first.,—
3,Acid,Poison,NaN,40,100,30,NaN,May lower opponent's Special Defense.,10
4,Acid Armor,Poison,NaN,—,—,20,NaN,Sharply raises user's Defense.,—
...,...,...,...,...,...,...,...,...,...
860,Yawn,Normal,NaN,—,—,10,NaN,Puts opponent to sleep in the next turn.,—
861,Zap Cannon,Electric,NaN,120,50,5,NaN,Paralyzes opponent.,100
862,Zen Headbutt,Psychic,NaN,80,90,15,NaN,May cause flinching.,20
863,Zing Zap,Electric,NaN,80,100,10,NaN,May cause flinching.,30


In [134]:
#print(move_json)

## Natures and berries (nature_name, Increases, Decreases, Berrie_like, berrie_dislike)

In [100]:
natures_URL = "https://pokemondb.net/mechanics/natures"
natures_page = requests.get(natures_URL)
natures_soup = BeautifulSoup(natures_page.content, parser = 'html.parser')
natures_df = pd.read_html(str(natures_soup))[1]
natures_df

,Nature,Increases,Decreases
0,Adamant,Attack,Sp. Atk
1,Bashful,Sp. Atk,Sp. Atk
2,Bold,Defense,Attack
3,Brave,Attack,Speed
4,Calm,Sp. Def,Attack
5,Careful,Sp. Def,Sp. Atk
6,Docile,Defense,Defense
7,Gentle,Sp. Def,Defense
8,Hardy,Attack,Attack
9,Hasty,Speed,Defense


In base alla natura si hanno bacche che il pokemon apprezza disprezzza

In [101]:
def berries_taste(x):
    if x == 'Attack':
        val = 'Spicy'
    elif x == 'Defense':
        val = 'Sour'
    elif x == 'Speed':
        val = 'Sweet'
    elif x == 'Sp. Atk':
        val = 'Dry'
    elif x == 'Sp. Def': 
        val = 'Bitter'
    return val

In [102]:
natures_df['Likes_berrie'] = natures_df['Increases'].apply(berries_taste)
natures_df['Dislikes_berrie'] = natures_df['Decreases'].apply(berries_taste)

In [103]:
natures_df

,Nature,Increases,Decreases,Likes_berrie,Dislikes_berrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry
2,Bold,Defense,Attack,Sour,Spicy
3,Brave,Attack,Speed,Spicy,Sweet
4,Calm,Sp. Def,Attack,Bitter,Spicy
5,Careful,Sp. Def,Sp. Atk,Bitter,Dry
6,Docile,Defense,Defense,Sour,Sour
7,Gentle,Sp. Def,Defense,Bitter,Sour
8,Hardy,Attack,Attack,Spicy,Spicy
9,Hasty,Speed,Defense,Sweet,Sour


In [104]:
natures_json = json.dumps(list(natures_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/natures.json', 'w', encoding='utf8') as outfile:
    outfile.write(natures_json)
# CSV
natures_df.to_csv('./Scraping_datasets/natures.csv', na_rep="NA")

## Type Chart

In [8]:
typechart_URL = "https://pokemondb.net/type"
typechart_page = requests.get(typechart_URL)
typechart_soup = BeautifulSoup(typechart_page.content, parser = 'html.parser')
typechart_df = pd.read_html(str(typechart_soup))[0]
typechart_df

,DEFENSE →ATTACK ↴,Nor,Fir,Wat,Ele,Gra,Ice,Fig,Poi,Gro,Fly,Psy,Bug,Roc,Gho,Dra,Dar,Ste,Fai
0,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,½,0,NaN,NaN,½,NaN
1,Fire,NaN,½,½,NaN,2,2,NaN,NaN,NaN,NaN,NaN,2,½,NaN,½,NaN,2,NaN
2,Water,NaN,2,½,NaN,½,NaN,NaN,NaN,2,NaN,NaN,NaN,2,NaN,½,NaN,NaN,NaN
3,Electric,NaN,NaN,2,½,½,NaN,NaN,NaN,0,2,NaN,NaN,NaN,NaN,½,NaN,NaN,NaN
4,Grass,NaN,½,2,NaN,½,NaN,NaN,½,2,½,NaN,½,2,NaN,½,NaN,½,NaN
5,Ice,NaN,½,½,NaN,2,½,NaN,NaN,2,2,NaN,NaN,NaN,NaN,2,NaN,½,NaN
6,Fighting,2.0,NaN,NaN,NaN,NaN,2,NaN,½,NaN,½,½,½,2,0,NaN,2,2,½
7,Poison,NaN,NaN,NaN,NaN,2,NaN,NaN,½,½,NaN,NaN,NaN,½,½,NaN,NaN,0,2
8,Ground,NaN,2,NaN,2,½,NaN,NaN,2,NaN,0,NaN,½,2,NaN,NaN,NaN,2,NaN
9,Flying,NaN,NaN,NaN,½,2,NaN,2,NaN,NaN,NaN,NaN,2,½,NaN,NaN,NaN,½,NaN


In [40]:
colnames = ['Atk_Type/Pokemon_Type'] + list(typechart_df.iloc[:, 0])
print(colnames)

['Atk_Type/Pokemon_Type', 'Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fighting', 'Poison', 'Ground', 'Flying', 'Psychic', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy']


In [41]:
typechart_df.columns = colnames

In [42]:
typechart_df

,Atk_Type/Pokemon_Type,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
0,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,½,0,NaN,NaN,½,NaN
1,Fire,NaN,½,½,NaN,2,2,NaN,NaN,NaN,NaN,NaN,2,½,NaN,½,NaN,2,NaN
2,Water,NaN,2,½,NaN,½,NaN,NaN,NaN,2,NaN,NaN,NaN,2,NaN,½,NaN,NaN,NaN
3,Electric,NaN,NaN,2,½,½,NaN,NaN,NaN,0,2,NaN,NaN,NaN,NaN,½,NaN,NaN,NaN
4,Grass,NaN,½,2,NaN,½,NaN,NaN,½,2,½,NaN,½,2,NaN,½,NaN,½,NaN
5,Ice,NaN,½,½,NaN,2,½,NaN,NaN,2,2,NaN,NaN,NaN,NaN,2,NaN,½,NaN
6,Fighting,2.0,NaN,NaN,NaN,NaN,2,NaN,½,NaN,½,½,½,2,0,NaN,2,2,½
7,Poison,NaN,NaN,NaN,NaN,2,NaN,NaN,½,½,NaN,NaN,NaN,½,½,NaN,NaN,0,2
8,Ground,NaN,2,NaN,2,½,NaN,NaN,2,NaN,0,NaN,½,2,NaN,NaN,NaN,2,NaN
9,Flying,NaN,NaN,NaN,½,2,NaN,2,NaN,NaN,NaN,NaN,2,½,NaN,NaN,NaN,½,NaN


In [44]:
typechart_json = json.dumps(list(typechart_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
# JSON
with open('./Scraping_datasets/pokemondb/typechart.json', 'w', encoding='utf8') as outfile:
    outfile.write(typechart_json)
# CSV
typechart_df.to_csv('./Scraping_datasets/pokemondb/typechart.csv', na_rep="NA")

# 2. `Pikalytics` Series 12 VGC 2022 (Apr 22)

## Selenium

- *Lista dei pokemon registrati e link alle specifiche pagine*

In [6]:
driver = webdriver.Edge('msedgedriver')

In [7]:
driver.get('https://pikalytics.com/pokedex/ss/')

In [199]:
driver.title

'VGC 2022 Series 12 Stats Pokédex | Pokémon | Pikalytics'

Seleziono la porzione di scermo di interesse

In [8]:
pokemon_list = driver.find_element(by = 'xpath', value = '//*[@id="pokedex_wrapper"]')

Scrollo in modo tale da mostrare tutti i pokemon nell'HTML

In [9]:
from selenium.webdriver.common.action_chains import ActionChains

Muovo il cursore sulla lista (non serve, funziona anche il tab da solo)

In [10]:
actions = ActionChains(driver)
actions.move_to_element(pokemon_list).perform()

Faccio tabbare un numero molto superiore al numero di pokemon in modo da assicurarmi che vengano mostrati tutti

In [11]:
for i in range(1,10000):
    actions.key_down(value = '\ue004').perform()

KeyboardInterrupt: 

Estraggo il HTMLResponse Object dalla pagina ora completamente caricata

In [99]:
pikalytics_page = driver.page_source

In [ ]:
driver.close()
driver.quit()

## bs4

- *Estraggo la lista dal HTTPResponse Object sottenuto con selenium*

In [102]:
pikalytics_soup = BeautifulSoup(pikalytics_page, features='html.parser')

In [135]:
#print(pikalytics_soup.prettify())

### Lista dei link alle pagine dei singoli pokemon

In [108]:
pikalytics_pkm_min_list_ul = pikalytics_soup.find(id = "min_list")

In [212]:
pikalytics_pkm_min_list_a = pikalytics_pkm_min_list_ul.find_all('a', recursive=False)

In [214]:
pikalytics_pkm_min_list_links = []

In [215]:
for i in pikalytics_pkm_min_list_a:
    pikalytics_pkm_min_list_links.append(i['href'])

In [136]:
#pikalytics_pkm_min_list_links

### Definizione Dataset Vuoti

In [430]:
usage_S12VGC_columns = ['Pokemon', 'Monthly Usage (k)', 'Usage Percent (%)', 'Monthly Rank']
usage_S12VGC = pd.DataFrame(columns = usage_S12VGC_columns)

basestats_S12VGC_columns = ['Pokemon', 'HP', 'Atk', 'Def', 'SpA', 'SpD', 'Spe']
basestats_S12VGC = pd.DataFrame(columns = basestats_S12VGC_columns)

moves_S12VGC_columns = ['Pokemon', 'Move', 'Type', 'Use_Percentage (%)']
moves_S12VGC = pd.DataFrame(columns = moves_S12VGC_columns)
                            
teammates_S12VGC_columns = ['Pokemon', 'Teammate', 'Use_Percentage (%)']
teammates_S12VGC = pd.DataFrame(columns = teammates_S12VGC_columns)
                            
items_S12VGC_columns = ['Pokemon', 'Item', 'Use_Percentage (%)']
items_S12VGC = pd.DataFrame(columns = items_S12VGC_columns)
                            
abilities_S12VGC_columns = ['Pokemon', 'Ability', 'Use_Percentage (%)']
abilities_S12VGC = pd.DataFrame(columns = abilities_S12VGC_columns)
                            
EV_spread_S12VGC_columns = ['Pokemon', 'Nature', 'HP/Atk/Def/SpA/SpD/Spe', 'Use_Percentage (%)']
EV_spread_S12VGC = pd.DataFrame(columns = EV_spread_S12VGC_columns)

### Scraping bs4

In [431]:
from IPython.display import clear_output
lista_dei_vuoti = []
for i, link in enumerate(pikalytics_pkm_min_list_links):
    
    #print(link)
    
    # Acquisizione soup pagina specifica
    
    pikalytics_pkm_URL = "https://pikalytics.com" + link
    pikalytics_pkm_page = requests.get(pikalytics_pkm_URL)
    pikalytics_pkm_soup = BeautifulSoup(pikalytics_pkm_page.content,
                                        features='html.parser')
    
    ## Controllo che la pagina esista verificando che compaia il nome del pokemon
    if (pikalytics_pkm_soup.find('h1') == None):
        lista_dei_vuoti.append(link)
        continue
    
    pikalytics_pkm_name = (pikalytics_pkm_soup.find('h1').text)

    # Avanzamento
    print(
        f'Avanzamento: {i}/{len(pikalytics_pkm_min_list_links)} - Pokemon: {pikalytics_pkm_name}'
    )

    # Usage Stats

    pikalytics_pkm_usagestats_temp = [f'{pikalytics_pkm_name}']
    pikalytics_pkm_usagestats_div = pikalytics_pkm_soup.find(
        'div', class_="header-div-right header-div-right-margin").find_all(
            'div', class_='pokemon-ind-summary-text')
    ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
    if (len(pikalytics_pkm_usagestats_div) < 3):
        pikalytics_pkm_usagestats_temp = [f'{pikalytics_pkm_name}', '-', '-', '-']
    else:
        for e, stat in enumerate(pikalytics_pkm_usagestats_div):
            if (e == 2):
                pikalytics_pkm_usagestats_temp.append(stat.contents[1])
            else:
                pikalytics_pkm_usagestats_temp.append(stat.contents[0])
    ## Trasformo riga in serie e appendo
    usagestats_series = pd.Series(pikalytics_pkm_usagestats_temp,
                                  index=usage_S12VGC.columns)
    usage_S12VGC = usage_S12VGC.append(usagestats_series, ignore_index=True)

    # Base Stats

    pikalytics_pkm_basestats_temp = [f'{pikalytics_pkm_name}']
    pikalytics_pkm_basestats_stat_wrapper = pikalytics_pkm_soup.find(
        'div', id='bstats_wrapper').find('div',
                                         recursive=False).find('div',
                                                               recursive=False)
    ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
    if pikalytics_pkm_basestats_stat_wrapper:
        pikalytics_pkm_basestats_stat_div = pikalytics_pkm_basestats_stat_wrapper.find_all(
            'div', recursive=False)
        for stat in pikalytics_pkm_basestats_stat_div:
            pikalytics_pkm_basestats_temp.append(
                stat.find_all('div', recursive=False)[1].text)
        ## Trasformo riga in serie e appendo
        base_stat_series = pd.Series(pikalytics_pkm_basestats_temp,
                                     index=basestats_S12VGC.columns)
        basestat_S12VGC = basestat_S12VGC.append(base_stat_series,
                                                 ignore_index=True)
    # Moves Stats

    pikalytics_pkm_moves_wrapper = pikalytics_pkm_soup.find(id="moves_wrapper")
    pikalytics_pkm_moves_div = pikalytics_pkm_moves_wrapper.find_all(
        'div', class_="pokedex-move-entry-new")
    for moves in pikalytics_pkm_moves_div:
        # In questo caso, essendoci più mosse consigliate per ogni pokemon, dobbiamo creare una lista per ogni mossa (una riga nel dataframe)
        pikalytics_pkm_moves_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not moves.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 3 variabili
        move = moves.find_all('div', recursive=False)
        pikalytics_pkm_moves_temp.append(move[0].text)
        pikalytics_pkm_moves_temp.append(move[1].find('span').text)
        pikalytics_pkm_moves_temp.append(move[2].text)
        ## Trasformo riga in serie e appendo
        moves_series = pd.Series(pikalytics_pkm_moves_temp,
                                 index=moves_S12VGC.columns)
        moves_S12VGC = moves_S12VGC.append(moves_series, ignore_index=True)

    # Teammates Stats

    pikalytics_pkm_teammates_wrapper = pikalytics_pkm_soup.find(
        id="dex_team_wrapper")
    pikalytics_pkm_teammates_div = pikalytics_pkm_teammates_wrapper.find_all(
        'a', class_="teammate_entry", recursive=False)
    for teammates in pikalytics_pkm_teammates_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_teammates_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not teammates.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 2 variabili
        pikalytics_pkm_teammates_temp.append(teammates['data-name'])
        pikalytics_pkm_teammates_temp.append(
            teammates.find_all('div', recursive=False)[3].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        teammates_series = pd.Series(pikalytics_pkm_teammates_temp,
                                     index=teammates_S12VGC.columns)
        teammates_S12VGC = teammates_S12VGC.append(teammates_series,
                                                   ignore_index=True)

    # Items Stats

    pikalytics_pkm_items_wrapper = pikalytics_pkm_soup.find(
        id="items_wrapper").find('div')
    pikalytics_pkm_items_div = pikalytics_pkm_items_wrapper.find_all(
        'div', recursive=False)
    for item in pikalytics_pkm_items_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_items_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not item.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 2 variabili
        pikalytics_pkm_items_temp.append(
            item.find_all('div', recursive=False)[1].text)
        pikalytics_pkm_items_temp.append(
            item.find_all('div', recursive=False)[2].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        items_series = pd.Series(pikalytics_pkm_items_temp,
                                 index=items_S12VGC.columns)
        items_S12VGC = items_S12VGC.append(items_series, ignore_index=True)
    
    # Abilities

    pikalytics_pkm_abilities_wrapper = pikalytics_pkm_soup.find(
        id="abilities_wrapper").find('div')
    pikalytics_pkm_abilities_div = pikalytics_pkm_abilities_wrapper.find_all(
        'div', recursive=False)
    for ability in pikalytics_pkm_abilities_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_abilities_temp = [f'{pikalytics_pkm_name}']
        # Vado ad aggiungere separatamente le 2 variabili
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not ability.find_all('div', recursive=False):
            break
        pikalytics_pkm_abilities_temp.append(
            ability.find_all('div', recursive=False)[0].text)
        pikalytics_pkm_abilities_temp.append(
            ability.find_all('div', recursive=False)[1].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        abilities_series = pd.Series(pikalytics_pkm_abilities_temp,
                                 index=abilities_S12VGC.columns)
        abilities_S12VGC = abilities_S12VGC.append(abilities_series, ignore_index=True)
        
    # ev_spread

    pikalytics_pkm_ev_spread_wrapper = pikalytics_pkm_soup.find(
        id="dex_spreads_wrapper").find('div')
    pikalytics_pkm_ev_spread_div = pikalytics_pkm_ev_spread_wrapper.find_all(
        'div', recursive=False)
    for ev_spread in pikalytics_pkm_ev_spread_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_ev_spread_temp = [f'{pikalytics_pkm_name}']
        # Vado ad aggiungere separatamente le 3 variabili
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not ev_spread.find_all('div', recursive=False):
            break
        ## Natura
        pikalytics_pkm_ev_spread_temp.append(
            ev_spread.find_all('div', recursive=False)[0].text)
        ## Spreads li vado a sommare per ottenere un'unica stat
        spread_div = ev_spread.find_all('div', recursive=False)[1:7]
        spread_temp = ""
        for ev in (ev_spread.find_all('div', recursive=False)[1:7]):
            spread_temp += ev.text
        pikalytics_pkm_ev_spread_temp.append(spread_temp)
        ## Percentage
        pikalytics_pkm_ev_spread_temp.append(
            ev_spread.find_all('div', recursive=False)[7].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_ev_spread_temp)
        ev_spread_series = pd.Series(pikalytics_pkm_ev_spread_temp,
                                 index=EV_spread_S12VGC.columns)
        EV_spread_S12VGC = EV_spread_S12VGC.append(ev_spread_series, ignore_index=True)
    clear_output(wait=True)
    #if (i > 5):
    #    break

Avanzamento: 634/662 - Pokemon: Gourgeist-Large


## Dataset Ottenuti

In [137]:
#lista_dei_vuoti

In [433]:
display(usage_S12VGC)

,Pokemon,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,Zacian-Crowned,1088,65,1
1,Incineroar,1124,59,2
2,Kyogre,744,35,3
3,Grimmsnarl,577,30,4
4,Regieleki,603,29,5
...,...,...,...,...
337,Steelix,-,-,-
338,Noctowl,-,-,-
339,Corsola,-,-,-
340,Mime Jr.,-,-,-


In [434]:
display(basestat_S12VGC)

,Pokemon,HP,Atk,Def,SpA,SpD,Spe
0,Zacian-Crowned,92,170,115,80,115,148
1,Grimmsnarl,95,120,65,95,75,60
2,Regieleki,80,100,50,100,50,200
3,Rillaboom,100,125,90,60,70,85
4,Calyrex-Shadow,100,85,80,165,100,150
...,...,...,...,...,...,...,...
1420,Steelix,75,85,200,55,65,30
1421,Noctowl,100,50,50,86,96,70
1422,Corsola,65,55,95,65,95,35
1423,Mime Jr.,20,25,45,70,90,60


In [435]:
display(moves_S12VGC)

,Pokemon,Move,Type,Use_Percentage (%)
0,Zacian-Crowned,Behemoth Blade,steel,99.996%
1,Zacian-Crowned,Protect,normal,98.672%
2,Zacian-Crowned,Sacred Sword,fighting,84.154%
3,Zacian-Crowned,Play Rough,fairy,64.487%
4,Zacian-Crowned,Substitute,normal,30.274%
...,...,...,...,...
3613,Mime Jr.,Other,,12.562%
3614,Gourgeist-Large,Power Whip,grass,100.000%
3615,Gourgeist-Large,Trick Room,psychic,99.986%
3616,Gourgeist-Large,Imprison,psychic,99.986%


In [436]:
display(teammates_S12VGC)

,Pokemon,Teammate,Use_Percentage (%)
0,Zacian-Crowned,Incineroar,54.189%
1,Zacian-Crowned,Kyogre,31.467%
2,Zacian-Crowned,Grimmsnarl,31.148%
3,Zacian-Crowned,Rillaboom,29.083%
4,Zacian-Crowned,Gastrodon,24.347%
...,...,...,...
3417,Gourgeist-Large,Weezing-Galar,0.014%
3418,Gourgeist-Large,Duraludon,0.014%
3419,Gourgeist-Large,Togetic,0.014%
3420,Gourgeist-Large,Talonflame,0.014%


In [437]:
display(items_S12VGC)

,Pokemon,Item,Use_Percentage (%)
0,Zacian-Crowned,Rusted Sword,100.000%
1,Incineroar,Shuca Berry,36.082%
2,Incineroar,Safety Goggles,23.390%
3,Incineroar,Sitrus Berry,14.031%
4,Incineroar,Focus Sash,8.994%
...,...,...,...
1887,Corsola,Other,0.734%
1888,Mime Jr.,Focus Sash,99.860%
1889,Mime Jr.,Other,0.140%
1890,Gourgeist-Large,Figy Berry,99.986%


In [438]:
display(abilities_S12VGC)

,Pokemon,Ability,Use_Percentage (%)
0,Zacian-Crowned,Intrepid Sword,100.000%
1,Incineroar,Intimidate,99.927%
2,Incineroar,Blaze,0.073%
3,Kyogre,Drizzle,100.000%
4,Grimmsnarl,Prankster,99.959%
...,...,...,...
768,Mime Jr.,Soundproof,99.870%
769,Mime Jr.,Filter,0.130%
770,Mime Jr.,Technician,0.000%
771,Gourgeist-Large,Frisk,100.000%


In [439]:
display(EV_spread_S12VGC)

,Pokemon,Nature,HP/Atk/Def/SpA/SpD/Spe,Use_Percentage (%)
0,Zacian-Crowned,Jolly,0/252/0/0/4/252,10.388%
1,Zacian-Crowned,Adamant,252/28/4/0/12/212,4.504%
2,Zacian-Crowned,Jolly,4/252/0/0/0/252,4.419%
3,Zacian-Crowned,Adamant,188/164/4/0/4/148,2.891%
4,Zacian-Crowned,Jolly,0/252/4/0/0/252,2.456%
...,...,...,...,...
6814,Gourgeist-Large,Sassy,252/4/0/0/252/0,0.000%
6815,Gourgeist-Large,Relaxed,252/0/252/4/0/0,0.000%
6816,Gourgeist-Large,Quiet,252/0/0/252/4/0,0.000%
6817,Gourgeist-Large,Adamant,252/60/0/0/196/0,0.000%


## Salvataggio .csv relazionali

In [440]:
usage_S12VGC.to_csv('./Scraping_datasets/pikalytics/usage_S12VGC.csv', na_rep="NA")
basestat_S12VGC.to_csv('./Scraping_datasets/pikalytics/basestat_S12VGC.csv', na_rep="NA")
moves_S12VGC.to_csv('./Scraping_datasets/pikalytics/moves_S12VGC.csv', na_rep="NA")
teammates_S12VGC.to_csv('./Scraping_datasets/pikalytics/teammates_S12VGC.csv', na_rep="NA")
items_S12VGC.to_csv('./Scraping_datasets/pikalytics/items_S12VGC.csv', na_rep="NA")
abilities_S12VGC.to_csv('./Scraping_datasets/pikalytics/abilities_S12VGC.csv', na_rep="NA")
EV_spread_S12VGC.to_csv('./Scraping_datasets/pikalytics/EV_spread_S12VGC.csv', na_rep="NA")

## Salvataggio .JSON dizionari

In [51]:
S12VGC_df = {}
S12VGC_json = {}
dataset_list = ['usage_S12VGC', 'basestat_S12VGC', 'moves_S12VGC', 'teammates_S12VGC', 'items_S12VGC', 'abilities_S12VGC', 'EV_spread_S12VGC']

for dataset_name in dataset_list:
    print(dataset_name)
    S12VGC_df[f'{dataset_name}'] = pd.read_csv(f'./Scraping_datasets/pikalytics/{dataset_name}.csv', na_values = 'NA').iloc[: , 1:]
    S12VGC_json[f'{dataset_name}'] = json.dumps(list(S12VGC_df[f'{dataset_name}'].agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
    with open(f'./Scraping_datasets/pikalytics/{dataset_name}.json', 'w', encoding='utf8') as outfile:
        outfile.write(S12VGC_json[f'{dataset_name}'])
    


usage_S12VGC
basestat_S12VGC
moves_S12VGC
teammates_S12VGC
items_S12VGC
abilities_S12VGC
EV_spread_S12VGC


# 3. `serebii.net` (Sword and Shield Pokédex: Galar Pokédex, Isle of Armor Pokédex, Crown Tundra Pokédex)

Il regolamento del VGC (che scrapo nel punto successivo), fa riferimento ai pokedex regionali specifici di Sword e Shield (oltre a quello nazionale preso al punto 1), ho deciso quindi di scaricarli, dato che comunque tutti questi pokemon sono opoi quelli utilizzati nel VGC 12.

- *Il pokédex nazionale è quello che cointiene TUTTI i pokemon usciti nelle diverse generazioni (in totale 905)*
- Principalmente serviranno per il matching nell'identificazione di quali pokemon assergnare alle etichette PErmitted, Restricted, Banned etc.

## Galar Pokédex

Url della risorsa, HTTPResponse Object e BeautifulSoupObject della pagina

In [193]:
swordandshield_URL = "https://www.serebii.net/swordshield/galarpokedex.shtml"
swordandshield_page = requests.get(swordandshield_URL)

In [194]:
swordandshield_soup = BeautifulSoup(swordandshield_page.content, features="html.parser")

In [199]:
swordandshield_df = pd.read_html(str(swordandshield_soup), header=1)[1]

In [202]:
swordandshield_df.drop(axis = 1, labels = ['Pic', 'Type'], inplace = True)
swordandshield_df

,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd
0,#001,Grookeyサルノリ,Overgrow Grassy Surge,50,65,50,40,40,65
1,#002,Thwackeyバチンキー,Overgrow Grassy Surge,70,85,70,55,60,80
2,#003,Rillaboomゴリランダー,Overgrow Grassy Surge,100,125,90,60,70,85
3,#004,Scorbunnyヒバニー,Blaze Libero,50,71,40,40,40,69
4,#005,Rabootラビフット,Blaze Libero,65,86,60,55,60,94
...,...,...,...,...,...,...,...,...,...
395,#396,Drakloakドロンチ,Clear Body Infiltrator Cursed Body,68,80,50,60,50,102
396,#397,Dragapultドラパルト,Clear Body Infiltrator Cursed Body,88,120,75,100,75,142
397,#398,Zacianザシアン,Intrepid Sword,92,130,115,80,115,138
398,#399,Zamazentaザマゼンタ,Dauntless Shield,92,130,115,80,115,138


`#` indica il numero nel pokedex specifico dell'ottava generazione
- Vedi: https://pokemondb.net/pokedex/national

### Sprite

In [224]:
swordandshield_sprites = swordandshield_soup.find_all('td', class_ = 'pkmnblock')

In [229]:
swordandshield_sprites_list = []
for swordandshield_sprite in swordandshield_sprites:
    swordandshield_sprites_list.append('https://www.serebii.net/' + swordandshield_sprite.find('img', class_ = 'stdsprite')['src'])

Aggiungo la colonna

In [230]:
swordandshield_df['sprite_url'] = swordandshield_sprites_list

In [231]:
swordandshield_df

,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd,sprite_url
0,#001,Grookeyサルノリ,Overgrow Grassy Surge,50,65,50,40,40,65,https://www.serebii.net//swordshield/pokemon/s...
1,#002,Thwackeyバチンキー,Overgrow Grassy Surge,70,85,70,55,60,80,https://www.serebii.net//swordshield/pokemon/s...
2,#003,Rillaboomゴリランダー,Overgrow Grassy Surge,100,125,90,60,70,85,https://www.serebii.net//swordshield/pokemon/s...
3,#004,Scorbunnyヒバニー,Blaze Libero,50,71,40,40,40,69,https://www.serebii.net//swordshield/pokemon/s...
4,#005,Rabootラビフット,Blaze Libero,65,86,60,55,60,94,https://www.serebii.net//swordshield/pokemon/s...
...,...,...,...,...,...,...,...,...,...,...
395,#396,Drakloakドロンチ,Clear Body Infiltrator Cursed Body,68,80,50,60,50,102,https://www.serebii.net//swordshield/pokemon/s...
396,#397,Dragapultドラパルト,Clear Body Infiltrator Cursed Body,88,120,75,100,75,142,https://www.serebii.net//swordshield/pokemon/s...
397,#398,Zacianザシアン,Intrepid Sword,92,130,115,80,115,138,https://www.serebii.net//swordshield/pokemon/s...
398,#399,Zamazentaザマゼンタ,Dauntless Shield,92,130,115,80,115,138,https://www.serebii.net//swordshield/pokemon/s...


### Salvataggio

In [232]:
swordandshield_json = json.dumps(list(swordandshield_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4)

Salvataggio

In [242]:
# JSON
with open('./Scraping_datasets/serebii/swordandshield_pokedex.json', 'w', encoding='utf8') as outfile:
    outfile.write(swordandshield_json)
# CSV
swordandshield_df.to_csv('./Scraping_datasets/serebii/swordandshield_pokedex.csv', na_rep="NA")

## Isle of Armor Pokédex

Url della risorsa, HTTPResponse Object e BeautifulSoupObject della pagina

In [234]:
isleofarmor_URL = "https://www.serebii.net/swordshield/isleofarmordex.shtml"
isleofarmor_page = requests.get(isleofarmor_URL)

In [235]:
isleofarmor_soup = BeautifulSoup(isleofarmor_page.content, features="html.parser")

In [236]:
isleofarmor_df = pd.read_html(str(isleofarmor_soup), header=1)[1]

In [237]:
isleofarmor_df.drop(axis = 1, labels = ['Pic', 'Type'], inplace = True)
isleofarmor_df

,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd
0,#001,Slowpokeヤドン,Gluttony Own Tempo Regenerator,90,65,65,40,40,15
1,#002,Slowbroヤドラン,Quick Draw Own Tempo Regenerator,95,100,95,100,70,30
2,#003,Slowkingヤドキング,Curious Medicine Own Tempo Regenerator,95,65,80,110,110,30
3,#004,Bunearyミミロル,Run Away Klutz Limber,55,66,44,44,56,85
4,#005,Lopunnyミミロップ,Cute Charm Klutz Limber,65,76,84,54,96,105
...,...,...,...,...,...,...,...,...,...
206,#207,Dittoメタモン,Limber Imposter,48,48,48,48,48,48
207,#208,Porygonポリゴン,Trace Download Analytic,65,60,70,85,75,40
208,#209,Porygon2ポリゴン２,Trace Download Analytic,85,80,90,105,95,60
209,#210,Porygon-ZポリゴンＺ,Adaptability Download Analytic,85,80,70,135,75,90


`#` indica il numero nel pokedex specifico dell'ottava generazione
- Vedi: https://pokemondb.net/pokedex/national

### Sprite

In [238]:
isleofarmor_sprites = isleofarmor_soup.find_all('td', class_ = 'pkmnblock')

In [239]:
isleofarmor_sprites_list = []
for isleofarmor_sprite in isleofarmor_sprites:
    isleofarmor_sprites_list.append('https://www.serebii.net/' + isleofarmor_sprite.find('img', class_ = 'stdsprite')['src'])

Aggiungo la colonna

In [240]:
isleofarmor_df['sprite_url'] = isleofarmor_sprites_list

In [241]:
isleofarmor_df

,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd,sprite_url
0,#001,Slowpokeヤドン,Gluttony Own Tempo Regenerator,90,65,65,40,40,15,https://www.serebii.net//swordshield/pokemon/s...
1,#002,Slowbroヤドラン,Quick Draw Own Tempo Regenerator,95,100,95,100,70,30,https://www.serebii.net//swordshield/pokemon/s...
2,#003,Slowkingヤドキング,Curious Medicine Own Tempo Regenerator,95,65,80,110,110,30,https://www.serebii.net//swordshield/pokemon/s...
3,#004,Bunearyミミロル,Run Away Klutz Limber,55,66,44,44,56,85,https://www.serebii.net//swordshield/pokemon/s...
4,#005,Lopunnyミミロップ,Cute Charm Klutz Limber,65,76,84,54,96,105,https://www.serebii.net//swordshield/pokemon/s...
...,...,...,...,...,...,...,...,...,...,...
206,#207,Dittoメタモン,Limber Imposter,48,48,48,48,48,48,https://www.serebii.net//swordshield/pokemon/s...
207,#208,Porygonポリゴン,Trace Download Analytic,65,60,70,85,75,40,https://www.serebii.net//swordshield/pokemon/s...
208,#209,Porygon2ポリゴン２,Trace Download Analytic,85,80,90,105,95,60,https://www.serebii.net//swordshield/pokemon/s...
209,#210,Porygon-ZポリゴンＺ,Adaptability Download Analytic,85,80,70,135,75,90,https://www.serebii.net//swordshield/pokemon/s...


### Salvataggio

In [244]:
isleofarmor_json = json.dumps(list(isleofarmor_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4)

Salvataggio

In [245]:
# JSON
with open('./Scraping_datasets/serebii/isleofarmor_pokedex.json', 'w', encoding='utf8') as outfile:
    outfile.write(isleofarmor_json)
# CSV
isleofarmor_df.to_csv('./Scraping_datasets/serebii/isleofarmor_pokedex.csv', na_rep="NA")

## The Crown Tundra PokéDex

Url della risorsa, HTTPResponse Object e BeautifulSoupObject della pagina

In [246]:
thecrowntundra_URL = "https://www.serebii.net/swordshield/thecrowntundradex.shtml"
thecrowntundra_page = requests.get(thecrowntundra_URL)

In [247]:
thecrowntundra_soup = BeautifulSoup(thecrowntundra_page.content, features="html.parser")

In [248]:
thecrowntundra_df = pd.read_html(str(thecrowntundra_soup), header=1)[1]

In [249]:
thecrowntundra_df.drop(axis = 1, labels = ['Pic', 'Type'], inplace = True)
thecrowntundra_df

,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd
0,#001,Snomユキハミ,Shield Dust Ice Scales,30,25,35,45,30,20
1,#002,Frosmothモスノウ,Shield Dust Ice Scales,70,65,60,125,90,65
2,#003,Woolooウールー,Fluffy Run Away Bulletproof,42,40,55,40,45,48
3,#004,Dubwoolバイウールー,Fluffy Steadfast Bulletproof,72,80,100,60,90,88
4,#005,Skwovetホシガリス,Cheek Pouch Gluttony,70,55,55,35,35,25
...,...,...,...,...,...,...,...,...,...
205,#206,Terrakionテラキオン,Justified,91,129,90,72,90,108
206,#207,Virizionビリジオン,Justified,91,90,72,90,129,108
207,#208,Glastrierブリザポス,Chilling Neigh,100,145,130,65,110,30
208,#209,Spectrierレイスポス,Grim Neigh,100,65,60,145,80,130


`#` indica il numero nel pokedex specifico dell'ottava generazione
- Vedi: https://pokemondb.net/pokedex/national

### Sprite

In [250]:
thecrowntundra_sprites = thecrowntundra_soup.find_all('td', class_ = 'pkmnblock')

In [251]:
thecrowntundra_sprites_list = []
for thecrowntundra_sprite in thecrowntundra_sprites:
    thecrowntundra_sprites_list.append('https://www.serebii.net/' + thecrowntundra_sprite.find('img', class_ = 'stdsprite')['src'])

Aggiungo la colonna

In [252]:
thecrowntundra_df['sprite_url'] = thecrowntundra_sprites_list

In [253]:
thecrowntundra_df

,No.,Name,Abilities,HP,Att,Def,S.Att,S.Def,Spd,sprite_url
0,#001,Snomユキハミ,Shield Dust Ice Scales,30,25,35,45,30,20,https://www.serebii.net//swordshield/pokemon/s...
1,#002,Frosmothモスノウ,Shield Dust Ice Scales,70,65,60,125,90,65,https://www.serebii.net//swordshield/pokemon/s...
2,#003,Woolooウールー,Fluffy Run Away Bulletproof,42,40,55,40,45,48,https://www.serebii.net//swordshield/pokemon/s...
3,#004,Dubwoolバイウールー,Fluffy Steadfast Bulletproof,72,80,100,60,90,88,https://www.serebii.net//swordshield/pokemon/s...
4,#005,Skwovetホシガリス,Cheek Pouch Gluttony,70,55,55,35,35,25,https://www.serebii.net//swordshield/pokemon/s...
...,...,...,...,...,...,...,...,...,...,...
205,#206,Terrakionテラキオン,Justified,91,129,90,72,90,108,https://www.serebii.net//swordshield/pokemon/s...
206,#207,Virizionビリジオン,Justified,91,90,72,90,129,108,https://www.serebii.net//swordshield/pokemon/s...
207,#208,Glastrierブリザポス,Chilling Neigh,100,145,130,65,110,30,https://www.serebii.net//swordshield/pokemon/s...
208,#209,Spectrierレイスポス,Grim Neigh,100,65,60,145,80,130,https://www.serebii.net//swordshield/pokemon/s...


### Salvataggio

In [254]:
thecrowntundra_json = json.dumps(list(thecrowntundra_df.agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4)

Salvataggio

In [255]:
# JSON
with open('./Scraping_datasets/serebii/thecrowntundra_pokedex.json', 'w', encoding='utf8') as outfile:
    outfile.write(thecrowntundra_json)
# CSV
thecrowntundra_df.to_csv('./Scraping_datasets/serebii/thecrowntundra_pokedex.csv', na_rep="NA")

# 4. `victoryroadvgc` 2022 Play! Pokémon Season Structure (Permitted, Restricted, Banned e Gigantamax pokemons)

Sulla pagina https://victoryroadvgc.com/2020-season/ è presente il regolamento completo della season 2022 di VGC.
In particolare estraggo:
   - Permitted Pokemon 
   - Restricted Pokemon (players may use two of the following Restricted Pokémon in their teams)
   - Banned Pokemon 
   - Gigantamax Pokemon
   
Nei primi 2 casi si farà riferimento al numero del **pokedex nazionale di ottava gen** -> il `#` nel dataset `pokedex` preso da `pokemon db` ma anche ai **pokedex regionali** -> quindi andranno poi integradi a dovere

## bs4 setup

In [186]:
victoryroadvgc_URL = 'https://victoryroadvgc.com/2020-season/'
victoryroadvgc_page = requests.get(victoryroadvgc_URL)

In [187]:
victoryroadvgc_soup = BeautifulSoup(victoryroadvgc_page.content, features='html.parser')

## Permitted e  Restricted

In [188]:
national_pokedex_rules = victoryroadvgc_soup.find_all("strong", string="National Pokédex:")

Permitted

In [189]:
national_pokedex_rules[0].parent.contents[1]

' #243–245, #252–260, #380–381, #480–482, #485, #488, #638–642, #645, #722–730, #785–788, #793–799 and #803–806'

Restricted

In [190]:
national_pokedex_rules[1].parent.contents[1]

' #150, #249-250, #382–384, #483–484, #487, #643–644, #646, #716–718, #789–792, #800, #888–890 and #898'

Definzione liste dei pokemon permessi nei vari pokedex

In [191]:
galar_permitted = list(range(1, 398))
isleofarmor_permitted = list(range(1, 211))
crowntundra_permitted = list(range(1, 210))
national_permitted = list(range(243, 246)) + list(range(252, 261)) + list(
    range(380, 382)) + list(range(480, 483)) + [485] + [488] + list(range(638, 643)) + [645] + list(
            range(722, 731)) + list(range(785, 789)) + list(range(793, 799)) + list(
                range(803, 807))

Definizione lista dei pokemon Restricted

In [192]:
national_restricted = [150] + list(range(249, 251)) + list(
    range(382, 385)) + list(range(483, 485)) + [487] + list(range(643, 645)) + [646] + list(
            range(716, 719)) + list(range(789, 793)) +[800] + list(
                range(888, 890)) + [898]

## Banned 

In [193]:
banned = ['Mew', 'Celebi', 'Jirachi', 'Victini', 'Keldeo (both forms)', 'Genesect (all forms)', 'Diancie', 'Volcanion', 'Magearna (both forms)', 'Marshadow', 'Zeraora', 'Meltan', 'Melmetal', 'Zarude (both forms)']

## Gigantamax Pokémon allowed 

In [194]:
gigantamax_allowed_soup = victoryroadvgc_soup.find("th", string = 'Gigantamax Pokémon allowed for Series 5, 7, 8, 9, 11 and 12').parent.parent.parent.find('tbody')

In [195]:
import re
gigantamax_allowed_list = re.split('(?=[A-Z])', gigantamax_allowed_soup.text)[1:-9]

In [196]:
gigantamax_allowed_list = gigantamax_allowed_list + ['Duraludon','Single Strike Style Urshifu','Rapid Strike Style Urshifu']

In [197]:
gigantamax_allowed_list

['Venusaur',
 'Charizard',
 'Blastoise',
 'Butterfree',
 'Pikachu',
 'Meowth',
 'Machamp',
 'Gengar',
 'Kingler',
 'Lapras',
 'Eevee',
 'Snorlax',
 'Garbodor',
 'Rillaboom',
 'Cinderace',
 'Inteleon',
 'Corviknight',
 'Orbeetle',
 'Drednaw',
 'Coalossal',
 'Flapple',
 'Appletun',
 'Sandaconda',
 'Toxtricity(both forms)',
 'Centiskorch',
 'Hatterene',
 'Grimmsnarl',
 'Alcremie(all forms)',
 'Copperajah',
 'Duraludon',
 'Single Strike Style Urshifu',
 'Rapid Strike Style Urshifu']

# 5. `Bulbapedia` (G-max moves e Gigantamax pokemon)

## G-max moves

In [49]:
gmax_moves_URL = "https://bulbapedia.bulbagarden.net/wiki/G-Max_Move"
gmax_moves_page = requests.get(gmax_moves_URL)
gmax_moves_soup = BeautifulSoup(gmax_moves_page.content, parser = 'html.parser')
gmax_moves_df = pd.read_html(str(gmax_moves_soup))[0]
gmax_moves_df

,Max Move,Gigantamax Pokémon,Gigantamax Pokémon.1,Type,Additional effect,Image
0,G-Max Vine Lash,NaN,Venusaur,Grass,Inflicts damage for four turns on non-Grass-ty...,NaN
1,G-Max Wildfire,NaN,Charizard,Fire,Inflicts damage for four turns on non-Fire-typ...,NaN
2,G-Max Cannonade,NaN,Blastoise,Water,Inflicts damage for four turns on non-Water-ty...,NaN
3,G-Max Befuddle,NaN,Butterfree,Bug,"Inflicts poison, paralysis, or sleep on all op...",NaN
4,G-Max Volt Crash,NaN,Pikachu,Electric,Paralyzes all opponents,NaN
5,G-Max Gold Rush,NaN,Meowth,Normal,Scatters coins on the ground that are picked u...,NaN
6,G-Max Chi Strike,NaN,Machamp,Fighting,"Pumps up the user and its allies, raising the ...",NaN
7,G-Max Terror,NaN,Gengar,Ghost,Prevents the opponent from escaping or being r...,NaN
8,G-Max Foam Burst,NaN,Kingler,Water,Lowers the Speed of all opponents by two stages,NaN
9,G-Max Resonance,NaN,Lapras,Ice,Lowers damage from both Physical and Special m...,NaN


In [50]:
gmax_moves_df.drop(axis=1, labels='Image', inplace=True)

In [51]:
gmax_moves_sprites = gmax_moves_soup.find('tbody').find_all('tr')
len(gmax_moves_sprites)

34

In [52]:
gmax_sprites = []
for gmax_moves_sprite in gmax_moves_sprites[1:]:
    #print(gmax_moves_sprite.find_all('img'))
    gmax_sprites.append('https:' + gmax_moves_sprite.find('img')['src'])
    #gmax_sprites.append(gmax_moves_sprite.find('td').find('a').find('img')['src'])

In [53]:
gmax_moves_df['image_url'] = gmax_sprites
gmax_moves_df.drop(axis=1, labels='Gigantamax Pokémon', inplace=True)
gmax_moves_df.rename(columns={'Gigantamax Pokémon.1':'Gigantamax Pokémon'}, inplace=True)

In [54]:
gmax_moves_df

,Max Move,Gigantamax Pokémon,Type,Additional effect,image_url
0,G-Max Vine Lash,Venusaur,Grass,Inflicts damage for four turns on non-Grass-ty...,https://archives.bulbagarden.net/media/upload/...
1,G-Max Wildfire,Charizard,Fire,Inflicts damage for four turns on non-Fire-typ...,https://archives.bulbagarden.net/media/upload/...
2,G-Max Cannonade,Blastoise,Water,Inflicts damage for four turns on non-Water-ty...,https://archives.bulbagarden.net/media/upload/...
3,G-Max Befuddle,Butterfree,Bug,"Inflicts poison, paralysis, or sleep on all op...",https://archives.bulbagarden.net/media/upload/...
4,G-Max Volt Crash,Pikachu,Electric,Paralyzes all opponents,https://archives.bulbagarden.net/media/upload/...
5,G-Max Gold Rush,Meowth,Normal,Scatters coins on the ground that are picked u...,https://archives.bulbagarden.net/media/upload/...
6,G-Max Chi Strike,Machamp,Fighting,"Pumps up the user and its allies, raising the ...",https://archives.bulbagarden.net/media/upload/...
7,G-Max Terror,Gengar,Ghost,Prevents the opponent from escaping or being r...,https://archives.bulbagarden.net/media/upload/...
8,G-Max Foam Burst,Kingler,Water,Lowers the Speed of all opponents by two stages,https://archives.bulbagarden.net/media/upload/...
9,G-Max Resonance,Lapras,Ice,Lowers damage from both Physical and Special m...,https://archives.bulbagarden.net/media/upload/...


In [64]:
gmax_moves_df.to_csv('./Scraping_datasets/bulbapedia/gmax_moves.csv', na_rep="NA")

## Gigantamax Pokemon

In [154]:
gigantamax_pokemon_URL = "https://bulbapedia.bulbagarden.net/wiki/Gigantamax"
gigantamax_pokemon_page = requests.get(gigantamax_pokemon_URL)
gigantamax_pokemon_soup = BeautifulSoup(gigantamax_pokemon_page.content, parser = 'html.parser')
gigantamax_pokemon_df = pd.read_html(str(gigantamax_pokemon_soup))[0]
gigantamax_pokemon_df_1 = pd.read_html(str(gigantamax_pokemon_soup))[1]

In [155]:
gigantamax_pokemon_df.drop(axis=1, labels='Before Gigantamax', inplace=True)
gigantamax_pokemon_df_1.drop(axis=1, labels='Before Gigantamax', inplace=True)

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [156]:
gigantamax_pokemon_df.columns = gigantamax_pokemon_df.columns.droplevel()
gigantamax_pokemon_df_1.columns = gigantamax_pokemon_df_1.columns.droplevel()
gigantamax_pokemon_df_1.rename(columns={'G-Max Move type':'G-Max Move Type'}, inplace=True)

In [157]:
gigantamax_pokemon_df.drop(axis=1, labels=['Image'], inplace=True)
gigantamax_pokemon_df_1.drop(axis=1, labels=['Image'], inplace=True)

In [158]:
frames = [gigantamax_pokemon_df,gigantamax_pokemon_df_1]
gigantamax_pokemon_df = pd.concat(frames).reset_index().drop(axis=1,labels='index')

In [159]:
gigantamax_pokemon_df

,Pokémon,Type,Height,G-Max Move,G-Max Move Type
0,Charizard,Fire Flying,"91'10""+(28.0+ m)",G-Max Wildfire,Fire
1,Butterfree,Bug Flying,"55'09""+(17.0+ m)",G-Max Befuddle,Bug
2,Pikachu,Electric,"68'11""+(21.0+ m)",G-Max Volt Crash,Electric
3,Meowth,Normal,"108'03""+(33.0+ m)",G-Max Gold Rush,Normal
4,Machamp,Fighting,"82'00""+(25.0+ m)",G-Max Chi Strike,Fighting
5,Gengar,Ghost Poison,"65'07""+(20.0+ m)",G-Max Terror,Ghost
6,Kingler,Water,"62'04""+(19.0+ m)",G-Max Foam Burst,Water
7,Lapras,Water Ice,"78'09""+(24.0+ m)",G-Max Resonance,Ice
8,Eevee,Normal,"59'01""+(18.0+ m)",G-Max Cuddle,Normal
9,Snorlax,Normal,"114'10""+(35.0+ m)",G-Max Replenish,Normal


In [160]:
gigantamax_pokemon_df.to_csv('./Scraping_datasets/bulbapedia/gigantamax_pkmn.csv', na_rep="NA")

# INTEGRAZIONE

## G-Max Moves e Gigantamax Pokemon (integrazione su Pokemon e Moves)

In [11]:
path = 'C:/Users/giorg/OneDrive - Università degli Studi di Milano-Bicocca/Laurea Magistrale - Data Science/directory_progetti/Data-Management-Project//'

### Importazione e manipolazione

In [12]:
path = 'C:\\Users\\giorg\\OneDrive - Università degli Studi di Milano-Bicocca\\Laurea Magistrale - Data Science\\directory_progetti\\Data-Management-Project\\Dataset integrati\\'
pokemon_i_df = pd.read_csv(f'{path}pokemon_integrati.csv', index_col=0).drop(axis=1, labels='index')

In [13]:
pokemon_i_df.head()

,#,Name,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Variant,Varieties,sprite_url
0,1,Bulbasaur,1,Common,NaN,False,Grass,Poison,318,45,49,49,65,65,45,NaN,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,NaN,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,NaN,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,Mega Venusaur,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,4,Charmander,1,Common,NaN,False,Fire,NaN,309,39,52,43,60,50,65,NaN,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...


In [14]:
col = pokemon_i_df.pop("Variant")

In [15]:
pokemon_i_df.insert(2, "Variant", col)

In [16]:
pokemon_i_df.head()

,#,Name,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,NaN,1,Common,NaN,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,NaN,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,NaN,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,Mega Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,4,Charmander,NaN,1,Common,NaN,False,Fire,NaN,309,39,52,43,60,50,65,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...


### Gigantamax: integrazione url sprite e integrazione al dataset principale pokemon

In [17]:
pokemon_gmax_df = pd.read_csv('./Scraping_datasets/bulbapedia/gigantamax_pkmn.csv', na_values=['NA', 'NaN'], index_col=0)

In [18]:
pokemon_gmax_df

,Pokémon,Type,Height,G-Max Move,G-Max Move Type
0,Charizard,Fire Flying,"91'10""+(28.0+ m)",G-Max Wildfire,Fire
1,Butterfree,Bug Flying,"55'09""+(17.0+ m)",G-Max Befuddle,Bug
2,Pikachu,Electric,"68'11""+(21.0+ m)",G-Max Volt Crash,Electric
3,Meowth,Normal,"108'03""+(33.0+ m)",G-Max Gold Rush,Normal
4,Machamp,Fighting,"82'00""+(25.0+ m)",G-Max Chi Strike,Fighting
5,Gengar,Ghost Poison,"65'07""+(20.0+ m)",G-Max Terror,Ghost
6,Kingler,Water,"62'04""+(19.0+ m)",G-Max Foam Burst,Water
7,Lapras,Water Ice,"78'09""+(24.0+ m)",G-Max Resonance,Ice
8,Eevee,Normal,"59'01""+(18.0+ m)",G-Max Cuddle,Normal
9,Snorlax,Normal,"114'10""+(35.0+ m)",G-Max Replenish,Normal


In [19]:
moves_gmax_df = pd.read_csv('./Scraping_datasets/bulbapedia/gmax_moves.csv', na_values=['NA', 'NaN'], index_col=0)

In [20]:
pokemon_gmax_sprite_df = moves_gmax_df[['Gigantamax Pokémon', 'image_url']]

In [21]:
pokemon_gmax_df = pd.merge(pokemon_gmax_df,
                           pokemon_gmax_sprite_df,
                           how='inner',
                           left_on='Pokémon',
                           right_on='Gigantamax Pokémon')

In [22]:
pokemon_gmax_df.drop(axis=1, labels='Gigantamax Pokémon', inplace=True)

In [23]:
pokemon_gmax_df.insert(1, 'Variant', ['Gigantamax'] * len(pokemon_gmax_df))

In [24]:
pokemon_gmax_df[['Type1','Type2']] = pokemon_gmax_df['Type'].str.split(" ", 1, expand=True)
pokemon_gmax_df.drop(axis=1, labels='Type', inplace=True)

In [25]:
pokemon_gmax_df = pokemon_gmax_df.rename(columns={'Pokémon':'Name', 'image_url':'sprite_url'})
pokemon_gmax_df = pokemon_gmax_df[['Name', 'Variant', 'sprite_url', 'Type1', 'Type2']] 

In [26]:
pokemon_gmax_df.insert(1, 'Generation', [8] * len(pokemon_gmax_df))

Faccio un merge parziale con il dataset completo dei pokemon per ottenere il numero del pokedex nazionale

In [27]:
pokemon_gmax_df = pd.merge(
    pokemon_gmax_df,
    pokemon_i_df[pokemon_i_df['Variant'].isnull()][['#', 'Name']],
    how='inner', on='Name')

Vado ad unire il dataset dei pokemon integrale e il dataset delle gigantamax

In [28]:
frame = [pokemon_i_df, pokemon_gmax_df]
pokemon_i_df = pd.concat(frame)

In [29]:
pokemon_i_df = pokemon_i_df.convert_dtypes()
pokemon_i_df.sort_values(by = ['#'], inplace=True)

In [30]:
pokemon_i_df.reset_index(inplace=True)

In [31]:
pokemon_i_df.drop(axis=1, labels='index', inplace=True)

In [32]:
pokemon_i_df

,#,Name,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,Mega Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur,Gigantamax,8,<NA>,<NA>,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,896,Glastrier,<NA>,8,Legendary,<NA>,False,Ice,<NA>,580,100,145,130,65,110,30,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...
1091,897,Spectrier,<NA>,8,Legendary,<NA>,False,Ghost,<NA>,580,100,65,60,145,80,130,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...
1092,898,Calyrex,<NA>,8,Legendary,<NA>,False,Psychic,Grass,500,100,80,80,80,80,80,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...
1093,898,Calyrex,Ice Rider,8,Legendary,<NA>,False,Psychic,Ice,680,100,165,150,85,130,50,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...


### G-max moves: integrazione gmax dell'API e gmax dello scraping (record linkage)

Dataset API

In [33]:
moves_gmax_API_df = pd.read_csv('./Dataset integrati/gmax_moves.csv').drop(axis=1, labels='Unnamed: 0')
moves_gmax_API_df.head()

,Name,Type,Power,Acc.,PP,Effect,Prob. (%),Cat.
0,G Max Befuddle,Bug,—,∞,5,"Butterfree-exclusive G-Max Move. Poisons, para...",100,—
1,G Max Cannonade,Water,—,∞,10,Blastoise-exclusive G-Max Move. Damages non-Wa...,—,—
2,G Max Centiferno,Fire,—,∞,5,Centiskorch-exclusive G-Max Move. Traps oppone...,100,—
3,G Max Chi Strike,Fighting,—,∞,5,Machamp-exclusive G-Max Move. Increases critic...,—,—
4,G Max Cuddle,Normal,—,∞,5,Eevee-exclusive G-Max Move. Infatuates opponents.,100,—


Dataset Scraping da bulbapedia

In [34]:
moves_gmax_df.head()

,Max Move,Gigantamax Pokémon,Type,Additional effect,image_url
0,G-Max Vine Lash,Venusaur,Grass,Inflicts damage for four turns on non-Grass-ty...,https://archives.bulbagarden.net/media/upload/...
1,G-Max Wildfire,Charizard,Fire,Inflicts damage for four turns on non-Fire-typ...,https://archives.bulbagarden.net/media/upload/...
2,G-Max Cannonade,Blastoise,Water,Inflicts damage for four turns on non-Water-ty...,https://archives.bulbagarden.net/media/upload/...
3,G-Max Befuddle,Butterfree,Bug,"Inflicts poison, paralysis, or sleep on all op...",https://archives.bulbagarden.net/media/upload/...
4,G-Max Volt Crash,Pikachu,Electric,Paralyzes all opponents,https://archives.bulbagarden.net/media/upload/...


#### Record Linkage

Setup

In [35]:
indexer = rl.Index()
indexer.full()

<Index>

In [36]:
candidates = indexer.index(moves_gmax_df, moves_gmax_API_df)

Comparing del nome

In [37]:
compare = rl.Compare()
compare.string('Max Move', 'Name', threshold=0.9, label='Name')
features = compare.compute(candidates, moves_gmax_df, moves_gmax_API_df)

Come atteso matchano tutte

In [38]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0      33
0.0    1056
dtype: int64

Estraggo i match potenziali

In [39]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(columns={'Name':'Score'})

Mergio i due datasets

In [40]:
moves_gmax_i_df = pd.merge(potential_matches, moves_gmax_df, left_on="level_0", right_index=True)
moves_gmax_i_df = pd.merge(moves_gmax_i_df, moves_gmax_API_df.drop(axis=1, labels='Type'), left_on="level_1", right_index=True)

In [41]:
moves_gmax_i_df.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Name', 'Cat.', 'Effect'], inplace=True)

Sistemo nomi colonne e sostituisco i - con i Nan

In [42]:
moves_gmax_i_df = moves_gmax_i_df.replace(['—'],np.nan)

### Tabella ponte g-max moves/gigantamax pokemon

Estraggo le colonne del dataset dell'api utili come ponte

In [43]:
ponte_gmax_gigantamax = moves_gmax_i_df[['Max Move','Gigantamax Pokémon']]

In [44]:
ponte_gmax_gigantamax['Gigantamax Pokémon'] = ponte_gmax_gigantamax['Gigantamax Pokémon'] + ' Gigantamax'
ponte_gmax_gigantamax.to_csv('./Dataset integrati 2.0/ponte_moves_pokemon_GMAX_MOVE.csv', index=False)

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
ponte_gmax_gigantamax

,Max Move,Gigantamax Pokémon
0,G-Max Vine Lash,Venusaur Gigantamax
1,G-Max Wildfire,Charizard Gigantamax
2,G-Max Cannonade,Blastoise Gigantamax
3,G-Max Befuddle,Butterfree Gigantamax
4,G-Max Volt Crash,Pikachu Gigantamax
5,G-Max Gold Rush,Meowth Gigantamax
6,G-Max Chi Strike,Machamp Gigantamax
7,G-Max Terror,Gengar Gigantamax
8,G-Max Foam Burst,Kingler Gigantamax
9,G-Max Resonance,Lapras Gigantamax


### G-max moves: sistemazione ds integrato di moves integrazione con il dataset delle g-max ottenuto dallo scraping + api

Importazione dataset originale integrato, sistemazione delle colonne doppie 

In [46]:
moves_i_df = pd.read_csv('./Dataset integrati/moves_integrati.csv',
                         index_col=0).drop(axis=1, labels='index')
moves_i_df = moves_i_df.drop(axis=1,
                             labels=['Type_x', 'Power_x', 'Accuracy',
                                     'PP_x']).rename(columns={
                                         'Type_y': 'Type',
                                         'Power_y': 'Power',
                                         'PP_y': 'PP'
                                     })

Sostituzione dei dati vuoti da '-' a Nan

In [47]:
moves_i_df = moves_i_df.replace(['—'],np.nan)

In [48]:
moves_i_df.head(2)

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%),Learned_by
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,NaN,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,NaN,"[{'name': 'lycanroc-midday', 'url': 'https://p..."


Dataset ottenuto dall'integrazione di API e scraping

In [49]:
moves_gmax_i_df.head(2)

,Max Move,Gigantamax Pokémon,Type,Additional effect,image_url,Power,Acc.,PP,Prob. (%)
0,G-Max Vine Lash,Venusaur,Grass,Inflicts damage for four turns on non-Grass-ty...,https://archives.bulbagarden.net/media/upload/...,NaN,∞,10,NaN
1,G-Max Wildfire,Charizard,Fire,Inflicts damage for four turns on non-Fire-typ...,https://archives.bulbagarden.net/media/upload/...,NaN,∞,10,100


Vado a rinominare le colonne in modo da permettere la concatenazione e droppo quelle inutili

In [50]:
moves_gmax_i_df = moves_gmax_i_df.drop(axis = 1, labels=['image_url', 'Gigantamax Pokémon'])
moves_gmax_i_df.rename(columns={'Max Move':'Name','Additional effect':'Effect'}, inplace=True)
# Aggiungo una colonna relativa alla generazione di inserimento delle g-max
moves_gmax_i_df.insert(1, 'Introducted_in', [8] * len(moves_gmax_i_df))
frame = [moves_i_df, moves_gmax_i_df]
moves_i_df = pd.concat(frame).reset_index().drop(axis=1, labels='index')
# Converto float errati in interi
moves_i_df.replace(['—'],np.nan)
moves_i_df = moves_i_df.convert_dtypes()

In [51]:
moves_i_df.head(3)

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%),Learned_by
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,<NA>,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,<NA>,"[{'name': 'lycanroc-midday', 'url': 'https://p..."
2,Acid,1,Poison,40,100,30,Special,May lower opponent's Special Defense.,10,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."


### Dataset Ottenuti

In [52]:
pokemon_i_df # Pokemon Integrato
ponte_gmax_gigantamax # Tabella ponte
moves_i_df # Mosse integrato

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%),Learned_by
0,Absorb,1,Grass,20,100,25,Special,User recovers half the HP inflicted on opponent.,<NA>,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,Accelerock,7,Rock,40,100,20,Physical,User attacks first.,<NA>,"[{'name': 'lycanroc-midday', 'url': 'https://p..."
2,Acid,1,Poison,40,100,30,Special,May lower opponent's Special Defense.,10,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."
3,Acid Armor,1,Poison,<NA>,<NA>,20,Status,Sharply raises user's Defense.,<NA>,"[{'name': 'tentacool', 'url': 'https://pokeapi..."
4,Acid Spray,5,Poison,40,100,20,Special,Sharply lowers opponent's Special Defense.,100,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."
...,...,...,...,...,...,...,...,...,...,...
816,G-Max Finale,8,Fairy,<NA>,∞,5,<NA>,Heals the user and its allies by 1/6 their max...,<NA>,<NA>
817,G-Max Steelsurge,8,Steel,<NA>,∞,5,<NA>,Scatters sharp spikes around the field Works l...,<NA>,<NA>
818,G-Max Depletion,8,Dragon,<NA>,∞,5,<NA>,Takes away 2 PP from the last move the target ...,<NA>,<NA>
819,G-Max One Blow,8,Dark,<NA>,∞,5,<NA>,Hits the target even if it is protected by a p...,<NA>,<NA>


## Pokemon: sistemazione ds integrato

### Sistemazione dei nomi dei pokemon

Creo l'identificativo 'Nome' che sarà la somma di Name e Variant e permetterà di identitficare unvicoamente il pokemon

In [53]:
pokemon_i_df.head()

,#,Name,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Venusaur,Mega Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur,Gigantamax,8,<NA>,<NA>,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...


Faccio dei controlli:
- Se non è una variante allora il suo nome coinciderà con il nome della specie
- Se Name è contenuto in variant allora il nome coinciderà con variant
- Altrimenti il nome è la somma di specie e variante

In [54]:
pokemon_complete_name = []
for index, row in pokemon_i_df.iterrows():
    if pd.isna(row['Variant']):
        pokemon_complete_name.append(row['Name'])
    elif row['Name'] in row['Variant']:
        pokemon_complete_name.append(row['Variant'])
    else:
        pokemon_complete_name.append(row['Name'] + ' ' + row['Variant'])

Converto il vecchio Name nel nuovo Specie

In [55]:
pokemon_i_df.rename(columns={'Name':'Species'}, inplace=True)

Inserisco la colonna relativa al nome completo

In [56]:
pokemon_i_df.insert(1, 'Name', pokemon_complete_name)

Rimuovo dalla variante il nome del pokemon

In [57]:
pokemon_variant_checked = []
for index, row in pokemon_i_df.iterrows():
    if pd.isna(row['Variant']):
        pokemon_variant_checked.append('Nan')
    elif row['Species'] in row['Variant']:
        pokemon_variant_checked.append(row['Variant'].replace(row['Species'], '').strip())
    else:
        pokemon_variant_checked.append(row['Variant'])

In [58]:
pokemon_i_df['Variant'] = pokemon_variant_checked
pokemon_i_df['Variant'] = pokemon_i_df['Variant'].replace(['Nan'],np.nan)
pokemon_i_df = pokemon_i_df.convert_dtypes()

### Sistemazione generazioni per alcune varianti

In [59]:
pokemon_i_df.loc[pokemon_i_df["Variant"] == "Mega", 'Generation'] = 6
pokemon_i_df.loc[pokemon_i_df["Variant"] == "Mega", 'Generation'] = 7
pokemon_i_df.loc[pokemon_i_df["Variant"] == "Mega", 'Generation'] = 8
pokemon_i_df.loc[pokemon_i_df["Variant"] == "Mega", 'Generation'] = 6

#pokemon_generation_checked = []
#for index, row in pokemon_i_df.iterrows():
#    if pd.isnull(row['Generation']):
#        pokemon_variant_checked.append('Nan')
#    elif pd.isnull(row['Variant']):
#        pokemon_variant_checked.append(row['Generation'])
#    elif row['Variant'] == 'Mega':
#        pokemon_generation_checked.append('6')
#    elif row['Variant'] == 'Alolan':
#        pokemon_generation_checked.append('7')
#    elif row['Variant'] == 'Galarian':
#        pokemon_generation_checked.append('8') 
#    elif row['Variant'] == 'Primal': #ArcheoGroudon e ArcheoKyogre
#        pokemon_generation_checked.append('6')
#    else:
#        pokemon_generation_checked.append(row['Generation'])

### Sistemazione evolved_from per la variante mega e gigantamax

Per le mega, gigantamax e primal evolves_from deve far rifermento alla specie (erroneamente nel dataset fa invece riferimento all'evoluzione precedente)

In [60]:
pokemon_i_df.loc[pokemon_i_df['Variant'] == 'Mega', 'Evolves_from'] = pokemon_i_df['Species']
pokemon_i_df.loc[pokemon_i_df['Variant'] == 'Gigantamax', 'Evolves_from'] = pokemon_i_df['Species']
pokemon_i_df.loc[pokemon_i_df['Variant'] == 'Primal', 'Evolves_from'] = pokemon_i_df['Species']

### Dataset Ottenuto

In [61]:
pokemon_i_df.head(5)

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...


## Type, Ponte Type Chart (MOVE_EFFECTIVENESS_ON_POKEMON), Ponte Pokemon-Type (IS_OF_TYPE), Ponte Move-Type (MOVES_IS_TYPE)

### Type Entità

In [62]:
types_i_df = pd.read_csv("./API_datasets/types.csv")[['Name']]
types_i_df['Generation'] = [1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,2,6,2,3]
types_i_df

,Name,Generation
0,normal,1
1,fighting,1
2,flying,1
3,poison,1
4,ground,1
5,rock,1
6,bug,1
7,ghost,1
8,steel,2
9,fire,1


In [63]:
types_i_df.to_csv('./Dataset integrati 2.0/df_types.csv', index=False)

### Ponte type Chart

In [109]:
ponte_type_chart = pd.read_csv("./Scraping_datasets/pokemondb/typechart.csv",
                               index_col=0)
ponte_type_chart = ponte_type_chart.convert_dtypes()
ponte_type_chart

,Atk_Type/Pokemon_Type,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
0,Normal,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,½,0,<NA>,<NA>,½,<NA>
1,Fire,<NA>,½,½,<NA>,2,2,<NA>,<NA>,<NA>,<NA>,<NA>,2,½,<NA>,½,<NA>,2,<NA>
2,Water,<NA>,2,½,<NA>,½,<NA>,<NA>,<NA>,2,<NA>,<NA>,<NA>,2,<NA>,½,<NA>,<NA>,<NA>
3,Electric,<NA>,<NA>,2,½,½,<NA>,<NA>,<NA>,0,2,<NA>,<NA>,<NA>,<NA>,½,<NA>,<NA>,<NA>
4,Grass,<NA>,½,2,<NA>,½,<NA>,<NA>,½,2,½,<NA>,½,2,<NA>,½,<NA>,½,<NA>
5,Ice,<NA>,½,½,<NA>,2,½,<NA>,<NA>,2,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,½,<NA>
6,Fighting,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,½,<NA>,½,½,½,2,0,<NA>,2,2,½
7,Poison,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,½,½,<NA>,<NA>,<NA>,½,½,<NA>,<NA>,0,2
8,Ground,<NA>,2,<NA>,2,½,<NA>,<NA>,2,<NA>,0,<NA>,½,2,<NA>,<NA>,<NA>,2,<NA>
9,Flying,<NA>,<NA>,<NA>,½,2,<NA>,2,<NA>,<NA>,<NA>,<NA>,2,½,<NA>,<NA>,<NA>,½,<NA>


Vado a normalizzare il dataset trasformandolo in due colonne

In [110]:
ponte_type_chart = pd.melt(
    ponte_type_chart,
    id_vars=['Atk_Type/Pokemon_Type'],
    value_vars=ponte_type_chart.columns.values.tolist()[1:],
    var_name='Def. Pokemon Type',
    value_name='Damage Multiplier').rename(
        columns={'Atk_Type/Pokemon_Type': 'Atk. Move Type'})

In [111]:
ponte_type_chart

,Atk. Move Type,Def. Pokemon Type,Damage Multiplier
0,Normal,Normal,<NA>
1,Fire,Normal,<NA>
2,Water,Normal,<NA>
3,Electric,Normal,<NA>
4,Grass,Normal,<NA>
...,...,...,...
319,Ghost,Fairy,<NA>
320,Dragon,Fairy,0
321,Dark,Fairy,½
322,Steel,Fairy,2


Aggiungo il parametro `effectiveness`

In [112]:
conditions = [
    ponte_type_chart['Damage Multiplier'].isnull(),
    ponte_type_chart['Damage Multiplier'] == '0',
    ponte_type_chart['Damage Multiplier'] == 0,
    ponte_type_chart['Damage Multiplier'] == '½',
    ponte_type_chart['Damage Multiplier'] == '2',
    ponte_type_chart['Damage Multiplier'] == 2
]

outputs = [
    'Normal (100%)', 'Normal (100%)', 'No effect (0%)',
    'Not very effective (50%)', 'Super-effective (200%)',
    'Super-effective (200%)'
]

ponte_type_chart['Effectiveness'] = np.select(conditions, outputs)
ponte_type_chart

,Atk. Move Type,Def. Pokemon Type,Damage Multiplier,Effectiveness
0,Normal,Normal,<NA>,Normal (100%)
1,Fire,Normal,<NA>,Normal (100%)
2,Water,Normal,<NA>,Normal (100%)
3,Electric,Normal,<NA>,Normal (100%)
4,Grass,Normal,<NA>,Normal (100%)
...,...,...,...,...
319,Ghost,Fairy,<NA>,Normal (100%)
320,Dragon,Fairy,0,Normal (100%)
321,Dark,Fairy,½,Not very effective (50%)
322,Steel,Fairy,2,Super-effective (200%)


In [114]:
ponte_type_chart.to_csv(
    './Dataset integrati 2.0/ponte_type_type_MOVE_EFFECTIVENESS_ON_POKEMON.csv',
    index=False)

### Ponte Pokemon-Type

Estraggo dal dataset dei pokemon i soli nomi e tipi

In [68]:
import warnings
warnings.filterwarnings(action='once')
ponte_pokemon_type = pokemon_i_df[['Name', 'Type1', 'Type2']]
ponte_pokemon_type['Type'] = ponte_pokemon_type[['Type1','Type2']].values.tolist()
ponte_pokemon_type.drop(axis=1, labels=['Type1', 'Type2'], inplace=True)
ponte_pokemon_type = ponte_pokemon_type.explode('Type')
ponte_pokemon_type = ponte_pokemon_type[~ponte_pokemon_type['Type'].isnull()]
ponte_pokemon_type

C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\giorg\Documents\venv\scrap37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Name,Type
0,Bulbasaur,Grass
0,Bulbasaur,Poison
1,Ivysaur,Grass
1,Ivysaur,Poison
2,Venusaur,Grass
...,...,...
1092,Calyrex,Grass
1093,Calyrex Ice Rider,Psychic
1093,Calyrex Ice Rider,Ice
1094,Calyrex Shadow Rider,Psychic


In [69]:
ponte_pokemon_type.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_type_IS_OF_TYPE.csv',
    index=False)

### Ponte MoveType-Type (MOVES_IS_TYPE)

In [70]:
ponte_move_type_MOVES_IS_TYPE = moves_i_df[['Name', 'Type']]
ponte_move_type_MOVES_IS_TYPE = ponte_move_type_MOVES_IS_TYPE[
    ~ponte_move_type_MOVES_IS_TYPE['Type'].isnull()]
ponte_move_type_MOVES_IS_TYPE

,Name,Type
0,Absorb,Grass
1,Accelerock,Rock
2,Acid,Poison
3,Acid Armor,Poison
4,Acid Spray,Poison
...,...,...
816,G-Max Finale,Fairy
817,G-Max Steelsurge,Steel
818,G-Max Depletion,Dragon
819,G-Max One Blow,Dark


In [71]:
ponte_move_type_MOVES_IS_TYPE.to_csv(
    './Dataset integrati 2.0/ponte_move_type_MOVES_IS_TYPE.csv',
    index=False)

## Pokemon: Sistemazione ABILITIES e creazione tabelle ponte: EVOLVES_FROM (pokemon-Pokemon), MAY_HAS (Pokemon-Ability), MAY_LEARN (Pokemon-Move), HAS_VARIANT (Pokemon-Pokemon)

### EVOLVES_FROM

In [72]:
ponte_pokemon_pokemon_EVOLVES_FROM = pokemon_i_df[
    ~pd.isnull(pokemon_i_df['Evolves_from'])][['Name', 'Evolves_from']]
ponte_pokemon_pokemon_EVOLVES_FROM.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_pokemon_EVOLVES_FROM.csv',
    index=False)

### HAS_VARIANT

In [73]:
pokemon_i_df.head(2)

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...


In [74]:
ponte_pokemon_pokemon_HAS_VARIANT = pd.merge(
    pokemon_i_df[pokemon_i_df['Variant'].isnull()][['Name', 'Species']],
    pokemon_i_df[~pokemon_i_df['Variant'].isnull()][['Name', 'Species']],
    on='Species',
    how='inner').rename(columns={
        'Name_x': 'Non-Variant Pkm Name',
        'Name_y': 'Variant Pkm Name'
    }).drop(axis=1, labels='Species')
ponte_pokemon_pokemon_HAS_VARIANT

,Non-Variant Pkm Name,Variant Pkm Name
0,Venusaur,Mega Venusaur
1,Venusaur,Venusaur Gigantamax
2,Charizard,Mega Charizard Y
3,Charizard,Charizard Gigantamax
4,Charizard,Mega Charizard X
...,...,...
147,Copperajah,Copperajah Gigantamax
148,Duraludon,Duraludon Gigantamax
149,Eternatus,Eternatus Eternamax
150,Calyrex,Calyrex Ice Rider


In [75]:
ponte_pokemon_pokemon_HAS_VARIANT.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_pokemon_HAS_VARIANT.csv',
    index=False)

### MAY_LEARN

#### Importazione e pulizia del dataset ponte MAY_LEARN

In [76]:
ponte_pokemon_moves_MAY_LEARN = pd.read_csv('./Tabelle ponte/ponte_move_poke.csv', index_col=0)
#ponte_pokemon_moves_MAY_LEARN

Pulizia stringhe

In [77]:
# Elimino trattini tra i nomi
ponte_pokemon_moves_MAY_LEARN['Pokemon'] = rl.preprocessing.clean(
    ponte_pokemon_moves_MAY_LEARN['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
ponte_pokemon_moves_MAY_LEARN['Pokemon'] = ponte_pokemon_moves_MAY_LEARN['Pokemon'].str.title()

In [78]:
ponte_pokemon_moves_MAY_LEARN.head(5)

,Move,Pokemon
0,Absorb,Zubat
1,Absorb,Golbat
2,Absorb,Oddish
3,Absorb,Gloom
4,Absorb,Vileplume


Faccio una serie di check:
- A quanto pare il matching serve farlo solo sui nomi dei pokemon (tutte e 788 le mosse della tabella ponte si trovano identicamente del dataset delle mosse)
- Mancano delle mosse (le G-Max) che però hanno la loro relazionbe specifica

In [79]:
moves_i_df.shape

(821, 10)

In [80]:
sum(moves_i_df['Name'].isin(ponte_pokemon_moves_MAY_LEARN['Move']))

788

In [81]:
len(ponte_pokemon_moves_MAY_LEARN['Move'].unique())

788

In [82]:
#moves_i_df[~moves_i_df['Name'].isin(ponte_pokemon_moves_MAY_LEARN['Move'])]['Name']

Procedo quindi con il matching tra i nomi dei pokemon

In [83]:
len(ponte_pokemon_moves_MAY_LEARN['Pokemon'].unique())

1093

Ci sono 1093 pokemon, 2 in meno rispetto a quello dei pokemon integrato

#### Record Linkage

In [130]:
pokemon_i_df

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,896,Glastrier,Glastrier,<NA>,8,Legendary,<NA>,False,Ice,<NA>,580,100,145,130,65,110,30,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...
1091,897,Spectrier,Spectrier,<NA>,8,Legendary,<NA>,False,Ghost,<NA>,580,100,65,60,145,80,130,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...
1092,898,Calyrex,Calyrex,<NA>,8,Legendary,<NA>,False,Psychic,Grass,500,100,80,80,80,80,80,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...
1093,898,Calyrex Ice Rider,Calyrex,Ice Rider,8,Legendary,<NA>,False,Psychic,Ice,680,100,165,150,85,130,50,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...


In [116]:
ponte_pokemon_moves_MAY_LEARN

,Move,Pokemon
0,Absorb,Zubat
1,Absorb,Golbat
2,Absorb,Oddish
3,Absorb,Gloom
4,Absorb,Vileplume
...,...,...
78177,Zen Headbutt,Calyrex Shadow
78178,Zing Zap,Togedemaru
78179,Zing Zap,Pincurchin
78180,Zing Zap,Togedemaru Totem


Essendo il dataset troppo grande procedo elaborando i soli valori unici, per poi generare una tabella ponte e ricongiungermi ai 781982 record iniziali

In [124]:
ponte_pokemon_moves_MAY_LEARN_unique = pd.DataFrame(ponte_pokemon_moves_MAY_LEARN['Pokemon'].unique(), columns=['Pokemon'])

In [125]:
ponte_pokemon_moves_MAY_LEARN_unique

,Pokemon
0,Zubat
1,Golbat
2,Oddish
3,Gloom
4,Vileplume
...,...
1088,Unown
1089,Blipbug
1090,Smeargle
1091,Cosmog


Setup

In [126]:
indexer = rl.Index()
indexer.full()

<Index>

In [127]:
candidates = indexer.index(ponte_pokemon_moves_MAY_LEARN_unique, pokemon_i_df)
len(candidates)

1196835

Comparing del nome

In [128]:
compare = rl.Compare()
compare.string('Pokemon', 'Name', threshold=0.85, label='Name', method='qgram')
features = compare.compute(candidates, ponte_pokemon_moves_MAY_LEARN_unique,
                           pokemon_i_df)

Come atteso matchano tutte

In [129]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0        940
0.0    1195895
dtype: int64

Estraggo i match potenziali

In [131]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(columns={'Name':'Score'})

In [132]:
potential_matches

,level_0,level_1,Score
0,0,59,1.0
1,1,60,1.0
2,2,61,1.0
3,3,62,1.0
4,4,63,1.0
...,...,...,...
935,1088,265,1.0
936,1089,999,1.0
937,1090,306,1.0
938,1091,956,1.0


Mergio i due datasets

In [133]:
ponte_pokemon_moves_MAY_LEARN_unique_ = pd.merge(potential_matches, ponte_pokemon_moves_MAY_LEARN_unique, left_on="level_0", right_index=True)
ponte_pokemon_moves_MAY_LEARN_unique_ = pd.merge(ponte_pokemon_moves_MAY_LEARN_unique_, pokemon_i_df[['Name', 'Variant']], left_on="level_1", right_index=True)

In [135]:
pd.set_option('display.max_rows', 30)
ponte_pokemon_moves_MAY_LEARN_unique_

,level_0,level_1,Score,Pokemon,Name,Variant
0,0,59,1.0,Zubat,Zubat,<NA>
1,1,60,1.0,Golbat,Golbat,<NA>
2,2,61,1.0,Oddish,Oddish,<NA>
3,3,62,1.0,Gloom,Gloom,<NA>
4,4,63,1.0,Vileplume,Vileplume,<NA>
5,5,64,1.0,Paras,Paras,<NA>
6,6,65,1.0,Parasect,Parasect,<NA>
7,7,146,1.0,Exeggcute,Exeggcute,<NA>
8,8,148,1.0,Exeggutor,Exeggutor,<NA>
9,9,161,1.0,Tangela,Tangela,<NA>


In [136]:
ponte_pokemon_moves_MAY_LEARN_unique_.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Variant'], inplace=True)
ponte_pokemon_moves_MAY_LEARN_unique_

,Pokemon,Name,Variant
0,Zubat,Zubat,<NA>
1,Golbat,Golbat,<NA>
2,Oddish,Oddish,<NA>
3,Gloom,Gloom,<NA>
4,Vileplume,Vileplume,<NA>
5,Paras,Paras,<NA>
6,Parasect,Parasect,<NA>
7,Exeggcute,Exeggcute,<NA>
8,Exeggutor,Exeggutor,<NA>
9,Tangela,Tangela,<NA>


Vado ora a joinare il datataset ponte con questa tabella ponte, rigenerando la tabella ponte originale con circa 78000 record

In [140]:
ponte_pokemon_moves_MAY_LEARN = pd.merge(ponte_pokemon_moves_MAY_LEARN, ponte_pokemon_moves_MAY_LEARN_unique_, on = 'Pokemon', how = 'inner')

Elimino la colonna con il nome non matchato (quella originale dell'api)

In [147]:
ponte_pokemon_moves_MAY_LEARN.drop(axis=1, labels='Pokemon', inplace=True)
ponte_pokemon_moves_MAY_LEARN.rename(columns={'Name':'Pokemon'}, inplace=True)

,Move,Pokemon
0,Absorb,Zubat
1,Acrobatics,Zubat
2,Aerial Ace,Zubat
3,Agility,Zubat
4,Air Cutter,Zubat
...,...,...
68100,Supersonic,Blipbug
68101,Sketch,Smeargle
68102,Splash,Cosmog
68103,Teleport,Cosmog


Salvataggio

In [185]:
ponte_pokemon_moves_MAY_LEARN.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_moves_MAY_LEARN.csv',
    index=False)

### ABILITIES

In [160]:
abilities_i_df = pd.read_csv('./Dataset integrati/Abilities_integrati.csv', index_col=0).drop(axis = 1, labels = ['index', 'Pokemon'])

In [161]:
abilities_i_df

,Name,Generation,Description
0,Adaptability,4,Powers up moves of the same type.
1,Aerilate,6,Turns Normal-type moves into Flying-type moves.
2,Aftermath,4,Damages the attacker landing the finishing hit.
3,Air Lock,3,Eliminates the effects of weather.
4,Analytic,5,Boosts move power when the Pokémon moves last.
...,...,...,...
262,White Smoke,3,Prevents other Pokémon from lowering its stats.
263,Wimp Out,7,Switches out when HP drops below half.
264,Wonder Guard,3,Only supereffective moves will hit.
265,Wonder Skin,5,Makes status-changing moves more likely to miss.


### MAY_HAS

#### Importazione e pulizia del dataset ponte MAY_LEARN e Abilities

In [152]:
ponte_pokemon_ability_MAY_HAS = pd.read_csv('./Tabelle ponte/ponte_abil_poke.csv', index_col=0)
ponte_pokemon_ability_MAY_HAS

,Ability,Pokemon,Hidden
0,Adaptability,eevee,False
1,Adaptability,corphish,True
2,Adaptability,crawdaunt,True
3,Adaptability,feebas,True
4,Adaptability,porygon-z,False
...,...,...,...
2528,Wonder Skin,bruxish,True
2529,Zen Mode,darmanitan-standard,True
2530,Zen Mode,darmanitan-zen,True
2531,Zen Mode,darmanitan-galar-standard,True


Pulizia stringhe

In [153]:
# Elimino trattini tra i nomi
ponte_pokemon_ability_MAY_HAS['Pokemon'] = rl.preprocessing.clean(
    ponte_pokemon_ability_MAY_HAS['Pokemon'])
# Rendo Maiuscola la prima lettera di ogni parola
ponte_pokemon_ability_MAY_HAS['Pokemon'] = ponte_pokemon_ability_MAY_HAS['Pokemon'].str.title()

In [154]:
ponte_pokemon_ability_MAY_HAS.head(5)

,Ability,Pokemon,Hidden
0,Adaptability,Eevee,False
1,Adaptability,Corphish,True
2,Adaptability,Crawdaunt,True
3,Adaptability,Feebas,True
4,Adaptability,Porygon Z,False


Faccio una serie di check:
- A quanto pare il matching serve farlo solo sui nomi dei pokemon (tutte e 788 le mosse della tabella ponte si trovano identicamente del dataset delle mosse)
- Mancano delle mosse (le G-Max) che però hanno la loro relazionbe specifica

In [162]:
abilities_i_df.shape

(267, 3)

In [163]:
sum(abilities_i_df['Name'].isin(ponte_pokemon_ability_MAY_HAS['Ability']))

267

In [164]:
len(ponte_pokemon_ability_MAY_HAS['Ability'].unique())

267

Ci sono 1093 pokemon, 2 in meno rispetto a quello dei pokemon integrato

#### Record Linkage

In [165]:
pokemon_i_df

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Varieties,sprite_url
0,1,Bulbasaur,Bulbasaur,<NA>,1,Common,<NA>,False,Grass,Poison,318,45,49,49,65,65,45,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,2,Ivysaur,Ivysaur,<NA>,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,3,Venusaur,Venusaur,<NA>,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,625,80,100,123,122,120,80,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,<NA>,Venusaur,<NA>,Grass,Poison,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,https://archives.bulbagarden.net/media/upload/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,896,Glastrier,Glastrier,<NA>,8,Legendary,<NA>,False,Ice,<NA>,580,100,145,130,65,110,30,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...
1091,897,Spectrier,Spectrier,<NA>,8,Legendary,<NA>,False,Ghost,<NA>,580,100,65,60,145,80,130,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...
1092,898,Calyrex,Calyrex,<NA>,8,Legendary,<NA>,False,Psychic,Grass,500,100,80,80,80,80,80,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...
1093,898,Calyrex Ice Rider,Calyrex,Ice Rider,8,Legendary,<NA>,False,Psychic,Ice,680,100,165,150,85,130,50,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...


In [166]:
ponte_pokemon_ability_MAY_HAS

,Ability,Pokemon,Hidden
0,Adaptability,Eevee,False
1,Adaptability,Corphish,True
2,Adaptability,Crawdaunt,True
3,Adaptability,Feebas,True
4,Adaptability,Porygon Z,False
...,...,...,...
2528,Wonder Skin,Bruxish,True
2529,Zen Mode,Darmanitan Standard,True
2530,Zen Mode,Darmanitan Zen,True
2531,Zen Mode,Darmanitan Galar Standard,True


Essendo il dataset troppo grande procedo elaborando i soli valori unici, per poi generare una tabella ponte e ricongiungermi ai 781982 record iniziali

In [167]:
ponte_pokemon_ability_MAY_HAS_unique = pd.DataFrame(ponte_pokemon_ability_MAY_HAS['Pokemon'].unique(), columns=['Pokemon'])

In [168]:
ponte_pokemon_ability_MAY_HAS_unique

,Pokemon
0,Eevee
1,Corphish
2,Crawdaunt
3,Feebas
4,Porygon Z
...,...
1121,Dewpider
1122,Araquanid
1123,Araquanid Totem
1124,Wimpod


Setup

In [169]:
indexer = rl.Index()
indexer.full()

<Index>

In [170]:
candidates = indexer.index(ponte_pokemon_ability_MAY_HAS_unique, pokemon_i_df)
len(candidates)

1232970

Comparing del nome

In [171]:
compare = rl.Compare()
compare.string('Pokemon', 'Name', threshold=0.85, label='Name', method='qgram')
features = compare.compute(candidates, ponte_pokemon_ability_MAY_HAS_unique,
                           pokemon_i_df)

Come atteso matchano tutte

In [172]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0        940
0.0    1232030
dtype: int64

Estraggo i match potenziali

In [173]:
potential_matches = features[features.sum(axis=1) > 0].reset_index().rename(columns={'Name':'Score'})

In [174]:
potential_matches

,level_0,level_1,Score
0,0,187,1.0
1,1,427,1.0
2,2,428,1.0
3,3,435,1.0
4,6,835,1.0
...,...,...,...
935,1120,879,1.0
936,1121,917,1.0
937,1122,918,1.0
938,1124,933,1.0


Mergio i due datasets

In [177]:
ponte_pokemon_ability_MAY_HAS_unique_ = pd.merge(potential_matches, ponte_pokemon_ability_MAY_HAS_unique, left_on="level_0", right_index=True)
ponte_pokemon_ability_MAY_HAS_unique_ = pd.merge(ponte_pokemon_ability_MAY_HAS_unique_, pokemon_i_df[['Name', 'Variant']], left_on="level_1", right_index=True)

In [178]:
pd.set_option('display.max_rows', 30)
ponte_pokemon_ability_MAY_HAS_unique_

,level_0,level_1,Score,Pokemon,Name,Variant
0,0,187,1.0,Eevee,Eevee,<NA>
1,1,427,1.0,Corphish,Corphish,<NA>
2,2,428,1.0,Crawdaunt,Crawdaunt,<NA>
3,3,435,1.0,Feebas,Feebas,<NA>
4,6,835,1.0,Skrelp,Skrelp,<NA>
...,...,...,...,...,...,...
935,1120,879,1.0,Volcanion,Volcanion,<NA>
936,1121,917,1.0,Dewpider,Dewpider,<NA>
937,1122,918,1.0,Araquanid,Araquanid,<NA>
938,1124,933,1.0,Wimpod,Wimpod,<NA>


In [179]:
ponte_pokemon_ability_MAY_HAS_unique_.drop(axis=1, labels=['level_0', 'level_1', 'Score', 'Variant'], inplace=True)
ponte_pokemon_ability_MAY_HAS_unique_

,Pokemon,Name
0,Eevee,Eevee
1,Corphish,Corphish
2,Crawdaunt,Crawdaunt
3,Feebas,Feebas
4,Skrelp,Skrelp
...,...,...
935,Volcanion,Volcanion
936,Dewpider,Dewpider
937,Araquanid,Araquanid
938,Wimpod,Wimpod


Vado ora a joinare il datataset ponte con questa tabella ponte, rigenerando la tabella ponte originale con circa 78000 record

In [180]:
ponte_pokemon_ability_MAY_HAS = pd.merge(ponte_pokemon_ability_MAY_HAS, ponte_pokemon_ability_MAY_HAS_unique_, on = 'Pokemon', how = 'inner')
ponte_pokemon_ability_MAY_HAS

,Ability,Pokemon,Hidden,Name
0,Adaptability,Eevee,False,Eevee
1,Anticipation,Eevee,True,Eevee
2,Run Away,Eevee,False,Eevee
3,Adaptability,Corphish,True,Corphish
4,Hyper Cutter,Corphish,False,Corphish
...,...,...,...,...
2170,Water Bubble,Dewpider,False,Dewpider
2171,Water Absorb,Araquanid,True,Araquanid
2172,Water Bubble,Araquanid,False,Araquanid
2173,Wimp Out,Wimpod,False,Wimpod


Elimino la colonna con il nome non matchato (quella originale dell'api)

In [181]:
ponte_pokemon_ability_MAY_HAS.drop(axis=1, labels='Pokemon', inplace=True)
ponte_pokemon_ability_MAY_HAS.rename(columns={'Name':'Pokemon'}, inplace=True)

In [182]:
ponte_pokemon_ability_MAY_HAS

,Ability,Hidden,Pokemon
0,Adaptability,False,Eevee
1,Anticipation,True,Eevee
2,Run Away,False,Eevee
3,Adaptability,True,Corphish
4,Hyper Cutter,False,Corphish
...,...,...,...
2170,Water Bubble,False,Dewpider
2171,Water Absorb,True,Araquanid
2172,Water Bubble,False,Araquanid
2173,Wimp Out,False,Wimpod


Salvataggio

In [183]:
ponte_pokemon_ability_MAY_HAS.to_csv(
    './Dataset integrati 2.0/ponte_pokemon_ability_MAY_HAS.csv',
    index=False)